ASTR 4260 Problem Set #5

 "First, let's import the modules that we will need"

In [1]:
import math
import numpy as np
import pylab as plot
%matplotlib inline

We can do the integration using SI units, defined below 
(or even better, use something like astropy)

In [2]:
G = 6.67e-11  # Gravitational constant [m^3/kg/s^2]
mass_solar = 1.99e30   # solar mass [kg]
mass_earth = 5.97e24   # mass of earth [kg]
mass_jupiter = 1.899e27  # mass of Jupiter [kg]
AU  = 1.496e11  # Astronomical unit [m] (earth-sun distance)

But instead, I find it much easier to define a set of dimensionless units which is equivalent to above but in units such that one year is 2*pi time units, and 1 AU is one length unit.

In [ ]:
G = 1.0
mass_solar = 1.0
mass_earth = 3.0e-6  # ratio of earth to sun mass
mass_jupiter = 0.000955   # ratio of jupiter to sun mass
AU = 1.0  # (earth-sun distance)

Next, let's define a function to compute the accelerations given the positions and masses. 
Note that we are assuming the positions are defined as a single two-dimension numpy array x[N,2] or x[N,3] for N bodies in 2 and 3 dimensions respectively.  Since this code uses the vector dot product to compute x^2, it is completely general.

In [ ]:
def ComputeAccel(x, m):
    """ returns gravitational acceleration given positions and masses """
    a = np.zeros_like(x)
    nbodies = x.shape[0]
    for i in range(nbodies):
        for j in range(nbodies):
            r = x[j]-x[i]
                if i != j:\n",
                    a[i] = a[i] + r*G*m[j]/pow(np.vdot(r, r), 1.5)
    return a

Now define the leapfrog integrator itself, which takes the positions x at time n and returns them at time n+1, and takes the velocites at time n+1/2 and returns them at time n+3/2.

In [ ]:
def LeapFrog(x, v, m, dt):
    """ performs one leap frog step, returns updated (x,v)"""
    x = x + dt*v
    a = ComputeAccel(x, m)
    v = v + dt*a
    return (x, v)

Now define a function which computes the potential and kinetic energies of the system.

In [ ]:
def CalculateEnergy(x, v, m):
   """ Computes kinetic and potential eneries of the system """
    KineticEnergy = 0.0
    PotentialEnergy = 0.0
    nbodies = x.shape[0]
    for i in range(nbodies):
        KineticEnergy += 0.5*m[i]*np.vdot(v[i], v[i])
        for j in range(nbodies):
                r = x[j]-x[i]
                if i != j:
                    PotentialEnergy -= 0.5*G*m[i]*m[j]/math.sqrt(np.vdot(r, r))
    return np.array([KineticEnergy, PotentialEnergy, KineticEnergy+PotentialEnergy])

Finally, we will need to set up the initial conditions.  Lots of ways to do this, but here we'll define a set of functions which adds a sun and earth (since we'll need these for problem 2 as well).

In [ ]:
# Set up the sun in the first spot of the position and velocity arrays.

def SetSunIC(x,v,m):
    x[0] = np.array([0.0, 0.0])
    v[0] = np.array([0.0, 0.0])
    m[0] = mass_solar
return

# Next, set up the earth in a circular orbit
def SetEarthIC(x,v,m):
    vorbit = math.sqrt(G*mass_solar/AU)
    torbit = 2.0*math.pi*AU/vorbit
    x[1] = np.array([1.0*AU, 0.0])
    v[1] = np.array([0.0, vorbit])
    m[1] = mass_earth\n
    return

Problem 1 - Earth/Sun system: 
Now let's apply the above to the earth-moon system.  First, compute some constants from our knowledge of circular orbits.

In [ ]:
torbit = 2.0*math.pi*AU/math.sqrt(G*mass_solar/AU) # orbital time
dt = torbit/50.0 # Arbitrarily set 50 points per orbit
tfinal = 5.0*torbit  # set number of years (earth orbits) to run
nsteps = int(tfinal/dt)  # number of steps"

Next, create the arrays to hold the positions, velocities and masses and set the initial conditions.

In [ ]:
nbodies = 2
x = np.zeros([nbodies,2])
v = np.zeros([nbodies,2])
m = np.zeros([nbodies])
SetSunIC(x,v,m)
SetEarthIC(x,v,m)

# Allocate arrays to hold the position and energy results

x_results = np.zeros([nsteps,nbodies,2])
E_results = np.zeros([nsteps,3])

# Before the main loop, take half a step to get velocities 1/2 step 
# in advance (for leap frog)"

a = ComputeAccel(x, m)
v = v + 0.5*dt*a"

# Loop over timesteps

for i in range(nsteps):
    (x, v) = LeapFrog(x, v, m, dt)\n
    x_results[i] = x\n
    E_results[i] = CalculateEnergy(x, v, m)


# As a simple check, compute final radius of earth as well as total energy and 
# compare to what they should be."

('Fractional radius change = ', 2.846123350197516e-05)
('Fractional energy change = ', -0.00014620792162756764)

print(\"Fractional radius change = \",(math.sqrt(np.vdot(x, x))-AU)/AU)
print(\"Fractional energy change = \",(E_results[-1,2] - E_results[0,2])/E_results[0,2])

# Plot results for the earth orbit"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAYYAAAD8CAYAAABzTgP2AAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADl0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uIDIuMi4zLCBodHRwOi8vbWF0cGxvdGxpYi5vcmcvIxREBQAAIABJREFUeJzt3Xu8VHW9//HXRxAwTe4oCgQqpaKFukUrU/N+6SdeCzMj09Dj7Zw6mnhURNREs6OWWqGoaCfxkh7RUFS89/PCxkREUhAxEBIEIRHk+jl/rLVbazYze89mZs93Lu/n4zGP9V1rfWf2e/aG+cy6fZe5OyIiIg02Cx1ARETKiwqDiIhkUGEQEZEMKgwiIpJBhUFERDKoMIiISAYVBhERyaDCICIiGVQYREQkQ9vQATZFt27dvG/fvqFjiIhUlKlTp37s7t2b61eRhaFv377U19eHjiEiUlHM7IN8+mlXkoiIZFBhEBGRDCoMIiKSQYVBREQyqDCIiEiGohQGM7vDzBaZ2Vs51puZ/drMZpvZm2a2Z2rdUDObFT+GFiOPiIhsumJtMdwFHNHE+iOB/vFjGPBbADPrAlwO7AMMAi43s85FyiQiIpugKNcxuPsLZta3iS6Dgbs9uo/oK2bWycx6AgcCT7n7UgAze4qowNxbjFwiRbNhAyybC+88Aa+Pg8V/a/lrfPkI2OdM6DUI2m0JZkWPKVIMpbrAbXtgXmp+frws13KR0nKHf34IEy+Edya2zs9494nokcvmW8L37oYdvg2btWmdDCJ5KFVhyPbVyJtYvvELmA0j2g1Fnz59ipdMatOyeXDjboW/zpY9oMsO0GFraNMO2mwOa1fBp/+AFR/Bpwvzf621n8EfTth4+aFXwr5nQ5uKHKhAKlCp/qXNB3qn5nsBC+LlBzZa/ly2F3D3McAYgLq6uqzFQySnj2fBzXX59z94BAwaBu2/2HqZINpF9f5z8NCZ8Nmi7H2euix6NNj3HDh0lAqFtBqLdvsX4YWiYwyPuftGX8PM7GjgXOAoogPNv3b3QfHB56lAw1lKrwN7NRxzyKWurs41VpI0ae0quONwWDit6X57nApH/RI236I0uVri76/CHYc13+/sV6DHLq2fRyqemU1192a/IRXlK4eZ3Uv0zb+bmc0nOtNocwB3/x0wkagozAZWAqfF65aa2ZXAlPilRjVXFERyWrsKrukFG9bl7nPiHbBblt015ajPPjByeTK/cincMADWrszsd+u+SfuUP0H/Q0qTT6pW0bYYSklbDPIv7jDhPPjrPdnX73EqHPOb6jwDaN5rMPbQ3Ot//j58oUvp8kjZK+kWg0jJLZ0Dv94j+7pjboY9Ty1tnhB6D0q2KFYtg2u/lLn+un7RdMBxcOKd1VkcpVWoMEhl+dMZMP2BjZfvdkK0m6hWbdEpKRKNi+aMh6MHaCtC8qLCIOVvw3oYlePDbPjfoUPH0uYpd112SIrEYz+D+rHJuoatiNMehy99o/TZpCLoGIOUr3Vr4KosdyHc5yw48trS56lkn8yFm7628fIh98LOR5U8joShYwxSuTZsgFFZhsw6/WnovXfp81SDzn2jrYj16+DKrsny8SdH0x9Pgj77Zn2q1B4VBikv1+0IKz/OXHb2q9Bj5zB5qk2btlGBcIcruyWn9t5xeDT96Qzo2CtcPikLKgxSHiZdAi/fnLnsJ8/A9nuFyVPtzGDEkqhAXNEpWX7DgGg6YqnGa6phKgwS1j/egt99M3PZCWNh9xPD5Kk1Ztl3MY3qAl37w3k6lleLVBgkjMbfVEGnnIbUsItp+fxkq2HJLBjZEYY+Cv32D5tPSkq39pTSmzpu46Jw+TIVhXLQsVdUII66Plk27v9FBaICz2CUTaPCIKXjHn3APHp+suy816MPIl2VW14G/SRznCaIivm7T4bJIyWlwiCl8d4zmVsJW3SOPni67hgukzRv5HI4N3Wc4Y8nRcVdqpoKg7S+kR3hnuOS+fP/ChfNDRZHWqhb/423HkZ2hE8/CpNHWp0Kg7SeNZ9t/O1y5PJoyAapPCOXww8eSuZ/9WWYPCpcHmk1KgzSOt58AH6xXTJ/wtiNv3VK5dnpYBjxSTL/4q90YLoKqTBI8V21DTx0RjJ/2ce6LqGabLZZVOS33T1ZdkWn6EZJUhWKUhjM7Agze8fMZpvZ8CzrbzCzN+LHu2a2LLVufWrdhGLkkYBGdoR1n6fml0ObzcPlkdZz1kvw49RZSldvC++/GC6PFE3BhcHM2gC3AEcCuwInm9mu6T7u/lN3H+juA4HfAKkdlaxqWOfuxxSaRwLZsD7zeMJBl2rXUS3osw9cujiZH/cdeOH63P2lIhRji2EQMNvd57j7GmA8MLiJ/icD9xbh50q5WP1p5v0SznwB9r8wXB4prbbtMr8EPHMl3HZQuDxSsGIUhu2Bean5+fGyjZjZl4B+wDOpxR3MrN7MXjGzY3P9EDMbFverX7x4ca5uUmofz4ZrUqNxDp8HPbOM+y/VL10cPpyq6x0qWDEKQ7ZLVnOdojAEeNDd16eW9YlvHPF94EYzy3rFk7uPcfc6d6/r3j3LzVuk9OZNgZtTo5+O+AQ6bB0uj4Q3cjnsdGhqXsWhEhWjMMwHeqfmewELcvQdQqPdSO6+IJ7OAZ4DctzhXcrK24/A2EOS+ZHLo7NVRH7wIHzrgmRexaHiFON/8hSgv5n1M7N2RB/+G51dZGZfAToDL6eWdTaz9nG7G/BN4O0iZJLW9Ob9cP8Pk3kdZJbGDr4MjvlNMq/iUFEKLgzuvg44F5gEzATud/cZZjbKzNJnGZ0MjPfMm0zvAtSb2TTgWWC0u6swlLPX74GHfpLMqyhILnv+EL57dzKv4lAxzCvwisW6ujqvr9cNREpu9mT4w/HJvIqC5OPdJ6PB9xro300wZjY1PqbbJO0Ulvx8PEtFQTbNlw+DIX9M5rXlUPZUGKR5q1fAzakvGSoK0lI7Hw3H/i6Zv/4r4bJIs1QYpGkbNsA1qctSVBRkUw08GfaOj0+t+AdMuiRsHslJhUGaNqpz0h6xNFwOqQ5HXw9bxtchvXwzzHstbB7JSoVBckvvC75wDmzWJlwWqR4Xzk7aYw+Fz7UVWm5UGCS7cakzjU99GLbsGi6LVJ/0LsnRfcLlkKxUGGRj7z0D7z8ftXc7EXbUgGjSCtK7JnWmUllRYZBMa1Zm3p/5xLHhskh126wNnP1qMv/7A8JlkQwqDJLpFz2Tts5AktbWY2fY56yovfANmK8LV8uBCoMk0pvzl3wULofUliOvTdq3Hwzr14XLIoAKgzRInzZ43BjYvEO4LFJ70lunV+pEh9BUGCQyNjWG/te+Fy6H1K7/fCdpP3tNuByiwiBk7kLScQUJ5YvbQu99ovbzo2H92rB5apgKQ617/8WkfcYzufuJlMLpTybtK7uFy1HjVBhq3bjvJO1ee+XuJ1IqF81N2q/dFixGLStKYTCzI8zsHTObbWbDs6z/kZktNrM34scZqXVDzWxW/BhajDySJ+1CknK0RWfoFF8NPfECqMB7xlS6gguDmbUBbgGOBHYFTjazXbN0vc/dB8aP2+PndgEuB/YBBgGXm1nnLM+VYluxOGl///5wOUSy+Y/pSfuKTuFy1KhibDEMAma7+xx3XwOMBwbn+dzDgafcfam7fwI8BRxRhEzSnOt3StpfPjxcDpFchj2XtD+ZGyhEbSpGYdgemJeanx8va+wEM3vTzB40s94tfK4U04u/StqXLQmXQ6Qp2+2RtG/6WrgcNagYhcGyLGu8U/BRoK+7fxV4GhjXgudGHc2GmVm9mdUvXrw4WxfJ1+RR0bTXIGjTNmwWkaaM+CRpv3xruBw1phiFYT7QOzXfC1iQ7uDuS9x9dTx7G7BXvs9NvcYYd69z97ru3bsXIXaNuvPopH3GU+FyiORjs82gZ7y1MOnisFlqSDEKwxSgv5n1M7N2wBBgQrqDmaVGZuMYYGbcngQcZmad44POh8XLpDW4wwcvRe1DrgibRSRfZ76QtO/7QbgcNaTgwuDu64BziT7QZwL3u/sMMxtlZg13eznfzGaY2TTgfOBH8XOXAlcSFZcpwKh4mbSGX+2ctPf7j3A5RFrqWxdE05mP6vTVEjCvwF9yXV2d19dreN4WcU9O+zv+dvjqSWHziLRUw3U3Ox4Mpz4UNkuFMrOp7l7XXD9d+Vwrru+ftFUUpBJ9+5Jo+t7ksDlqgApDrfgsPpPreA0xIBXqgJ8n7T//Z7gcNUCFoRb8cUjS/up3w+UQKdTAU6LplNvD5qhyKgy14N3Ho+kBF4XNIVKoY1PXMsx8NFyOKqfCUO2mjkva3/6vcDlEiqVNu2iqU1dbjQpDtXv0/GjauV/YHCLFcumipL1iUe5+sslUGKrZytQlIef/NVwOkWKy1Eg66bPtpGhUGKrZdamtBMs2LJVIhbpgdtKuwGuxyp0KQy047/XQCUSKa6vUeGkPDQuXo0qpMFSrmY8l7a47hssh0lp2jy/UnK4bTRWbCkO1uu+U0AlEWtcJqWsZ1q3O3U9aTIWhGqX3uV6qe1dIDbhx99AJqooKQzV66Yak3bZduBwire3wa6Lpio/C5qgyKgzVaHJ8r4WOfcLmEGltXz87aa9dFS5HlVFhqDbp3Uj//ka4HCKldvuhoRNUDRWGavNm6gyNzdqEyyFSKl8/N5p+ND1sjipSlMJgZkeY2TtmNtvMhmdZ/zMze9vM3jSzyWb2pdS69Wb2RvyY0Pi50kIP65xuqTGHX520N2wIl6OKFFwYzKwNcAtwJLArcLKZ7dqo21+BOnf/KvAgcF1q3Sp3Hxg/jkGK46K5oROIlN7rd4VOUBWKscUwCJjt7nPcfQ0wHhic7uDuz7r7ynj2FaBXEX6uNLZmZdLeonO4HCKhPPbT0AmqQjEKw/bAvNT8/HhZLqcDj6fmO5hZvZm9YmbH5nqSmQ2L+9UvXqxz87O684jQCUTCSI+dJAVrW4TXyDY6W9ZRrczsB0AdcEBqcR93X2BmOwDPmNl0d39voxd0HwOMAairq9OoWdksnBZNdzokbA6RUkuPneSuQSMLVIwthvlA79R8L2BB405mdghwCXCMu//r+nV3XxBP5wDPAXsUIVNtG3Jv6AQi4fzlptAJKl4xCsMUoL+Z9TOzdsAQIOPsIjPbA/g9UVFYlFre2czax+1uwDeBt4uQqfZsWJ+0dbWz1LKnLw+doOIVXBjcfR1wLjAJmAnc7+4zzGyUmTWcZfRLYCvggUanpe4C1JvZNOBZYLS7qzBsivo7QicQCeuMZ0InqBrFOMaAu08EJjZaNiLVzrrT293/P6DRr4ph4gWhE4iE1Wuv0Amqhq58rjYnjQudQCS8z/8ZOkFFU2GoNrsObr6PSLV7+ZbQCSqaCkO10Wl6IvD86NAJKpoKQzX4cGroBCLl4cjrmu8jzVJhqAZ357xgXKS27H1G6ARVQYWhGqyOD7S17xg2h0hoGmq+KFQYqsl3/jt0ApHyoTu6bTIVhmqy83dCJxApH0s0sN6mUmGoJpt3CJ1ApHzMnhw6QcVSYRCR6vTamNAJKpYKQ6XTrQxFMm23ZzT954dhc1QwFYZKt3Zl831EaslXjgydoOKpMFS6lUtCJxApL32/FTpBxVNhqHRzXwqdQKS89Ng5dIKKp8JQ6d6Z2HwfkVrSoVPoBBWvKIXBzI4ws3fMbLaZDc+yvr2Z3Revf9XM+qbWXRwvf8fMDi9Gnpoy77XQCUTKiwaSLFjBhcHM2gC3AEcCuwInm9mujbqdDnzi7jsBNwDXxs/dlehWoAOAI4Bb49eTfH22qPk+IiItUIwthkHAbHef4+5rgPFA45sCDAYa7iDzIHCwmVm8fLy7r3b394HZ8euJiEggxSgM2wPzUvPz42VZ+8T3iF4OdM3zuSIiUkLFKAzZduh5nn3yeW70AmbDzKzezOoXL17cwogiIpKvYhSG+UDv1HwvYEGuPmbWFugILM3zuQC4+xh3r3P3uu7duxchtoiIZFOMwjAF6G9m/cysHdHB5AmN+kwAhsbtE4Fn3N3j5UPis5b6Af0BnWbTEm3ah04gIlWmbaEv4O7rzOxcYBLQBrjD3WeY2Sig3t0nAGOBe8xsNtGWwpD4uTPM7H7gbWAdcI67ry80U03puhMsmhE6hYhUkYILA4C7TwQmNlo2ItX+HDgpx3OvBq4uRo6atM0AFQaRNM96mFJaQFc+V7odDwqdQKS8rP40dIKKp8JQ6XrtHTqBSHn5eFboBBVPhaHSfaFL6AQi5eWDv4ROUPFUGCrdFp1DJxApLzMbnxQpLaXCUOk0YJhIpvlToukXuobNUcFUGESkOtWdHjpBxVJhqCbr14VOIFI+9jgldIKKpcJQTWY9GTqBSPno2Cd0goqlwlBN/qRNZ5F/2Uwfb5tKv7lq0Cn+ZrR2ZdgcIqHpqueiUGGoBt/7Q+gEIuXh7y+HTlAVVBiqwTa7hU4gUh4eOSd0gqqgwlANNtNtskUAWDonmn7jvLA5KpwKQ7WZNyV0ApHw9vtZ6AQVTYWh2ow9JHQCkfA0hlhBVBiqxcAfhE4gEtY/s94VWDZBQYXBzLqY2VNmNiuebjSim5kNNLOXzWyGmb1pZt9LrbvLzN43szfix8BC8tS07/x36AQiYd2nL0fFUugWw3Bgsrv3BybH842tBH7o7gOAI4AbzaxTav2F7j4wfrxRYJ7a1TZ17+cNG8LlEAnlw6nR9KBLw+aoAoUWhsHAuLg9Dji2cQd3f9fdZ8XtBcAioHuBP1ea8qzulCo1TAeeC1ZoYdjG3RcCxNMeTXU2s0FAO+C91OKr411MN5hZ+xxPlXxY/Od88fqwOURKbc1nSVunbxes2cJgZk+b2VtZHoNb8oPMrCdwD3Cauzfs67gY2BnYG+gCXNTE84eZWb2Z1S9evLglP7p2XKBbGkqNuu/U0AmqStvmOrh7zvMfzewjM+vp7gvjD/5FOfptDfwZuNTdX0m99sK4udrM7gQuaCLHGGAMQF1dnQZEyWbLbkl79Qpov1W4LCKl9N7kaHqYdqMWQ6G7kiYAQ+P2UOCRxh3MrB3wMHC3uz/QaF3PeGpExyfeKjCPNLimV+gEIqW379mhE1SFQgvDaOBQM5sFHBrPY2Z1ZnZ73Oe7wP7Aj7Kclvo/ZjYdmA50A64qMI8cckXc0EaV1Ih3nkjaGmq7KMwrcJjauro6r6+vDx2jPG3YAKPiy0kuX6Z7Qkv1G9kx1V4eLkcFMLOp7l7XXD+V12qT/sb0v/8WLodIqV00N3SCqqHCUI12PDiaTrs3bA6R1jbr6aS9xUYDL8gmUmGoRt+/P2lX4K5Ckbz9zwmhE1QlFYZq1CZ1FvKDPw6XQ6RU/ksD6BWTCkO12v2kaDrjobA5RFrLpEuSdrstw+WoQioM1er425L2qk/C5RBpLS/fHE130j1Iik2FoVqlT1O9tm+wGCKtYu2qpH3Kg+FyVCkVhmp2vkYxlyp19bZJW9fqFJ0KQzXr0i9p33N8uBwireWsl0InqEoqDNXugPjeSQ2DjIlUusdT9wPbdvdwOaqYCkO1+/bFSfstnaEkVeDV30bTAceFzVHFVBhqQft4LJkHTwubQ6RQb6YGaD7prmAxqp0KQy0Y/kHSXjQzXA6RQj10RjT9Qrem+0lBVBhqQfqsjVv3DZdDpBAzH03aF84Ol6MGqDDUisuWJO2lc8LlENlU9/0gmm6+pU5RbWUqDLUiPX7Sr/cIl0NkU7x+T9L+rw/D5agRBRUGM+tiZk+Z2ax4mnXcWzNbn7p724TU8n5m9mr8/Pvi24BKa7nko6Q95/lwOURaasK50bTLjtpaKIFCtxiGA5PdvT8wOZ7PZpW7D4wfx6SWXwvcED//E+D0AvNIUzbvkLTvPiZ3P5FyMvHnSfv818PlqCGFFobBwLi4PQ44Nt8nmpkBBwENA5206PmyiS5flrT/95xwOUTy9drvo+nXvh82Rw0ptDBs4+4LAeJpjxz9OphZvZm9YmYNH/5dgWXuvi6enw9sn+sHmdmw+DXqFy9eXGDsGmYGe/8kar/xB93IR8pb+n7Ox/02XI4a02xhMLOnzeytLI/BLfg5feIbUH8fuNHMdgSy7SjM+Snl7mPcvc7d67p3796CHy0bOfr6pH1Fp3A5RJqy+tOkffL4cDlqULOFwd0PcffdsjweAT4ys54A8XRRjtdYEE/nAM8BewAfA53MrOF0mV6AbsNUKme+kLSXvBcuh0gu1/RK2l85MlyOGlTorqQJwNC4PRR4pHEHM+tsZu3jdjfgm8Db7u7As8CJTT1fWknPryXt3+wZLodINs9clbQvzfp9U1pRoYVhNHComc0CDo3nMbM6M7s97rMLUG9m04gKwWh3fztedxHwMzObTXTMYWyBeaQl0geixx4eLodI2vp18MIvo3a//aFt+7B5apB5BR58rKur8/r6+tAxqsNzo+G5a6L28HnQYeuweUTSB5xHLg+XowqZ2dT4eG+TdOVzrTswdenJ6N7hcogAPDUiaV/0Qe5+0qpUGCTziugbdeMTCWTNSvjLTVH7S/vBFjpjLhQVBomuiD5kZNRe9ndYOC1kGqlVv+iZtE/7c7gcosIgsf1+mrR/v78ufJPSunq7pH2pLmANTYVBEumzlHThm5TKe8/C2s+i9tG/grYaSzM0FQZJmMHP30/mr+0XLovUhrWfwz2pIdL2PiNcFvkXFQbJ9IUucHB8ZsiqpfDXP4TNI9Xt6m2Stk5NLRsqDLKxb/1n0n7kHFihK0+lFaSvV9DVzWVFhUGyS397u74/bNgQLotUn3RRGPa8rm4uMyoMklv6PtGjst6cT6TlJpyftA+6FLYbGC6LZKXCILm1aQsXzU3m09/yRDbFtPHwenxvr679Yf8Lw+aRrFQYpGlbdIYfpS42UnGQTfXhVHj4zGT+PI13Vq5UGKR5ffeDwbcm8yoO0lLL/g63HZTM6wyksqbCIPnZ45TMzX4VB8nXisWZY3CpKJQ9FQbJ30GXwj7/lsyrOEhzPvsYrt8pmU9fXS9lq6DCYGZdzOwpM5sVTzc6dcXMvm1mb6Qen5vZsfG6u8zs/dQ6nZ5Q7o4cDXsOTeZVHCSXTz6AX+6YzI/4JLq6XspeoVsMw4HJ7t4fmBzPZ3D3Z919oLsPBA4CVgJPprpc2LDe3d8oMI+UwjG/hr1/ksyrOEhj86bATV9N5kd8AptpB0WlKPQvNRiIzz1jHHBsE30hur/z4+6+ssCfK6EdfT0ceHEyr+IgDaaOg7GHJPOXL1NRqDCF/rW2cfeFAPG0RzP9hwD3Nlp2tZm9aWY3mJkuf6wkBw6PRsNsMLKjhuuudX86Ax5NXcB2+TLtPqpAzRYGM3vazN7K8hjckh9kZj2B3YFJqcUXAzsDewNdgIuaeP4wM6s3s/rFizVee9nY+wwY+mgyf0UnWPNZuDwSzsiOMP2B1PxyFYUK1WxhcPdD3H23LI9HgI/iD/yGD/6mRsL6LvCwu69NvfZCj6wG7gQGNZFjjLvXuXtd9+7d831/Ugr99ofzU4eHfrEdzP1LuDxSeuldiT0H6pTUClforqQJQMMpKkOBR5roezKNdiOliooRHZ94q8A8EkqXfjBiaTJ/11Ew7phweaQ0Vq/ILApH/wrOfD5cHimKQgvDaOBQM5sFHBrPY2Z1ZnZ7Qycz6wv0Bhr/i/kfM5sOTAe6AVcVmEdC2qxN5jfF95/XQelq9vYEuGb7ZP6cKbrRTpUwr8CDhXV1dV5fr3FWytoj52Te5OenM6Bjr3B5pLgaF/zLlkSDLkpZM7Op7l7XXD+dQyatY/AtcPYryfwNA+ChM3P3l8qwcunGRWHkchWFKqPCIK2nxy6ZQyC8OT76UNmwPlwm2XTjT4HrUvcBP2mcDjJXKRUGaV1m0YfHgOOTZaO6wCu/DZdJWmbtqqig/+2xZNllS2BAc9ezSqVSYZDSOOlOuGB2Mv/E8OjDZt3qcJmkefeeDFdvm8zvdZp2HdUA/XWldLbqHn2o3LIPLP5btOyqHtGdvHTTlvKyfH50XCjtko9g8w5h8khJaYtBSu+cV2H4vGR+yaxo6+Gdx8NlksiGDdHfIl0U9j0nKugqCjVDWwwSRoetow+b+jvhsf+Ilt07JJpeOAe27BouW6363bfgH29mLtOoqDVJ1zFIeO7RGEuNXfYxtNm89HlqzaP/DlPvylw27DnYbo8AYaQ15Xsdg7YYJLyGM5fWrY6OOTS4sls0HbE0uqpaiuupEfCXmzKXHX4NfP3sMHmkbKgwSPlo2z4qEMvmwY27JctHdYmmF38I7bcKk62aPHIu/PWezGV1P4bv3BAmj5QdFQYpP516RwViwRsw5oBkecO4PKc/Db33DpOtUrnDdTvAqqWZywccByfdFSSSlC8VBilf28XDN/9zAfz3LsnyhruD9T8MTnkg+3Ml8vFsuHmvjZdrC0GaoIPPUjnWr4Mrc5ytdN7r0HXH7OtqzYb1MLoPrFmx8brjfg9fG1L6TFIWdPBZqk+btsnYPJNHwYup24r+Zs+kfdFc2KJzSaMF5w4PnxWNR5XN8L9DBw2BLvnRFoNUtpVLMwd2a2zYc9V72uWGDTD+ZHj3iezrB98Ke5xS2kxS1rTFILXhC12SrYg5z8Pdje4aN+bApL3dnvDjJ6KznyrVu5Pgj9/NvX7AcXDCHbooTQpS0BaDmZ0EjAR2AQa5e9av8WZ2BHAT0Aa43d0b7vTWDxgPdAFeB0519zXN/VxtMUizlryXuXspm8594bQnYOueJYnUYuvXwhMXw5Tbmu632wlwwtjoehCRJuS7xVBoYdgF2AD8HrggW2EwszbAu0S3/pwPTAFOdve3zex+4CF3H29mvwOmuXuz4zGrMEiLrF8Hf/oxvN3ULclT9vsp7PezaNiOUnCHRTPh8Z/D3Bfze86w56OztkRaoCS7ktx9ZvzDmuo2CJjt7nPivuOBwWY2EzgI+H7cbxzR1ocG6pfiatMWvnt3Mr96BfzheJj3avb+L90QPZrS91vQa2/osSt06gNbdoP2W0PbdtE3/XWfw6plsPQ9mPtSdH/kFf/Y9Pdw7lTottOmP1+kBUpxjGF7IDWUJvOBfYCuwDJ3X5davj0ira39VnDBN4WpAAAFXUlEQVT6k5nL1q2Jhod49qr8XmPui/l/u2+pnzwbHTDXriEJpNnCYGZPA9tmWXWJu+ezbZ7tX7c3sTxXjmHAMIA+ffrk8WNFWqBtOzjgwujR2OpPYfqD8NptsGhGYT+nxwCoOw12HQxb9Wi+v0gAzRYGdz+kwJ8xH+idmu8FLAA+BjqZWdt4q6Fhea4cY4AxEB1jKDCTSP7afzH6MK87LXQSkZIoxTltU4D+ZtbPzNoBQ4AJHh31fhY4Me43FMjz6KCIiLSWggqDmR1nZvOBrwN/NrNJ8fLtzGwiQLw1cC4wCZgJ3O/uDdvjFwE/M7PZRMccxhaSR0RECqcrn0VEakS+p6vq8kgREcmgwiAiIhlUGEREJIMKg4iIZFBhEBGRDBV5VpKZLQY+KOJLdiO64K6S6T2Uj2p4H3oP5aHY7+FL7t69uU4VWRiKzczq8zmFq5zpPZSPangfeg/lIdR70K4kERHJoMIgIiIZVBgiY0IHKAK9h/JRDe9D76E8BHkPOsYgIiIZtMUgIiIZarIwmNlJZjbDzDaYWc4j/mY218ymm9kbZlZWo/a14D0cYWbvmNlsMxteyozNMbMuZvaUmc2Kp51z9Fsf/w3eMLMJpc6ZTXO/VzNrb2b3xetfNbO+pU/ZvDzex4/MbHHq939GiJy5mNkdZrbIzN7Ksd7M7Nfx+3vTzPYsdcbm5PEeDjSz5am/wYhWD+XuNfcAdgG+AjwH1DXRby7QLXTeTX0PQBvgPWAHoB0wDdg1dPZUvuuA4XF7OHBtjn4rQmdt6e8VOBv4XdweAtwXOvcmvo8fATeHztrEe9gf2BN4K8f6o4DHie4YuS/waujMm/AeDgQeK2WmmtxicPeZ7v5O6ByFyPM9DAJmu/scd18DjAcGt366vA0GxsXtccCxAbO0RD6/1/R7exA42KzsbuJc7v8+muXuLwBLm+gyGLjbI68Q3TWyZ2nS5SeP91ByNVkYWsCBJ81sanzP6UqzPTAvNT8/XlYutnH3hQDxNNdNkDuYWb2ZvWJm5VA88vm9/quPRzerWk50M6pyku+/jxPi3TAPmlnvLOvLWbn/H8jX181smpk9bmYDWvuHNXvP50plZk8D22ZZdYm753sL0W+6+wIz6wE8ZWZ/i6t7SRThPWT7hlrS09Caeg8teJk+8d9hB+AZM5vu7u8VJ+Emyef3Gvx3n4d8Mj4K3Ovuq83sLKKtoINaPVnxVMLfoTmvEw1lscLMjgL+F+jfmj+waguDux9ShNdYEE8XmdnDRJveJSsMRXgP84H0N7xewIICX7NFmnoPZvaRmfV094Xx5v2iHK/R8HeYY2bPAXsQ7RsPJZ/fa0Of+WbWFuhIme0uII/34e5LUrO3AdeWIFcxBf8/UCh3/2eqPdHMbjWzbu7eauNAaVdSDma2pZl9saENHAZkPWugjE0B+ptZPzNrR3QQtCzO6olNAIbG7aHARltBZtbZzNrH7W7AN4G3S5Ywu3x+r+n3diLwjMdHEstIs++j0f74Y4ju215JJgA/jM9O2hdY3rD7slKY2bYNx6fMbBDR5/aSpp9VoNBH5EM8gOOIvkmsBj4CJsXLtwMmxu0diM7SmAbMINp9Ezx7S95DPH8U8C7RN+xyew9dgcnArHjaJV5eB9wet78BTI//DtOB00PnzvV7BUYBx8TtDsADwGzgNWCH0Jk38X1cE//7nwY8C+wcOnOj/PcCC4G18f+H04GzgLPi9QbcEr+/6TRxFmIZv4dzU3+DV4BvtHYmXfksIiIZtCtJREQyqDCIiEgGFQYREcmgwiAiIhlUGEREJIMKg4iIZFBhEBGRDCoMIiKS4f8A0Hc21hwQpQQAAAAASUVORK5CYII=\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "plot.plot(x_results[:,0,0], x_results[:,0,1], label='sun')\n",
    "plot.plot(x_results[:,1,0], x_results[:,1,1], label='earth')\n",
    "plot.axis('equal')\n",
    "plot.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Plot evolution of energy against time"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 16,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAa0AAAEKCAYAAAChTwphAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADl0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uIDIuMi4zLCBodHRwOi8vbWF0cGxvdGxpYi5vcmcvIxREBQAAIABJREFUeJzt3XmYFPW97/H3t7tnmAEUFIkxogEVo6wjjCSGqLgBnkQxLrku56hEEJeYRJ9rhJhcYsxJTOTRROPGdYkYEzSaROIOLid4osKgKIoiqJxI9LqggsAMM939vX9UddMzdM/0DNPTU/B5PU8/U1X9q99S1V3f+lXX1M/cHRERkSiIlbsCIiIixVLQEhGRyFDQEhGRyFDQEhGRyFDQEhGRyFDQEhGRyFDQEhGRyFDQEhGRyFDQEhGRyEiUuwLbm912280HDhxY7mqIiETKkiVLPnL3/m2lU9DqZAMHDqSurq7c1RARiRQz+59i0unyoIiIRIaCloiIRIaCloiIRIaCloiIRIaCloiIRIaCloiIRIaCloiIRIb+T6ubuPMfq1lX30SPRIzKzCse/O2RiNEjEae6Mk51RZyelXGqwr89KxP0SMSIxaws9U6m0mxqSlHfmGJTY4pNjcmc6WB+U2Pw/uZkqmA+PRJxelQEba2qiAdtDv8GbU7QszLYBj0r41Ql4mVrM0BjMk19Y4r6pi1tbGgK2lyfsz2C6STJtGMYMQMzMDPMoCIWo6oyTlUiRnXYrupw/1Zn93Gcnj0SVFfEiZexze5OYypNQ2OaTU3JbPsbmlLUN6az26IhbH99U5qGphRmEDcjFjPiMctOV1XEgn1akci2s7qyxb6uiJOIl/fcOpX27D7NtDkz32yfN6VoCKdTaQ/aGjNiFuz3eMyoDD/Pmc9xbttzv9NVFTHMyruvN+d8xrPtb+VvQ1OKi4/ev+TfSwWtbuKu5/6HVR9s6PD6mS9C7heiKuegl5kO3k8QN8Nx0h58QN0h7U4y7dkvY0P4od2cDA9MmS9o45a/jal0J26F9rc5N5BVVybo2WJZz8oE1ZVxKuIx8n2VMl/OhqYUm5Pp8JVic1OahuSWtjbktL2+KUUy7V3eXoAeiRi9wgDWq8eWNmeme1VuCXoxg5gZFh40Y+FBsDGZpjGVbv43fGXanLu/G3IOTOVodmUiCG49K7YEterKOL1ypjOf7dwgEbQ7ODlIpj3bxqbctqfSbG5KU58Nwulse4MAnC7LZ9yMsL1bglrPFm3PvFeZiOXd12n3bFubUsHnvClnf2cDbc5nO7Pf65tSeDv3dczg/HH70rOytGHFvL01k1bV1tZ6R56I4WHAyB5Awg9X9iCaTGfP4gqdzeceZHIPsFsdeJua93gs84EHEnGjuiI46G15xbI9gMzZb8/K5l+ofGeOLc+aqxJx8p08Bl8uZ3MYKDfnBJDslygbKJN5e3GbGlNhj6/FsnC7NKXyf87NyPZkqyqCv0EvL5jeEujznQ1v/X7uyUPmAFNdEScRMzzcz2kHJzhRaEqlaQgPlA3Zg0g65ww2bO/m5m3eGPZoN4bbZOPmYN2Nm4PlDclU9kSkZaCJGc168pm/FfFYsxOf6rC3V9Wih5/7flXOSUJmPveKQI9EDHdIuZNKO+nM3zQ0JPP3zjNtbrasMRm2dcs2aLZu+NnObN90eCKWa0s7LXs1oyLefD+33Ke5Pd6qnG2S3RY5y6qzn/MY8ZiRSjspD9qa9mA600PPfA9z255tW4srF5tavt/iSkYy5eF+3npfJ2JGRTy3rVva3/JEt1nbw/3ec6vtkKC6MhamTeSkDT5H29I7NLMl7l7bVjr1tLoJM6MiHnzAevUobVnptOOQPRvtDnokoHePrv84unsXb4PmZVVVxNmpqvSlZoOle5dfbjODGEZFvPnyPlSUtNzcKwjxmHX5Zz0Rt7IcYD0neJXzcnKpKGjtgMr5W1B3012CdqmZGXGDlkFze5b53TD/heHt15Z9vX3S3YMiIhIZCloiIhIZCloiIhIZCloiIhIZCloiIhIZCloiIhIZCloiIhIZCloiIhIZCloiIhIZCloiIhIZCloiIhIZCloiIhIZCloiIhIZCloiIhIZCloiIhIZZQ1aZjbRzFaY2Sozm57n/R5mdk/4/vNmNjDnvRnh8hVmNqGtPM1sUJjHyjDPytbKMLN+ZvaUmW0ws9+WbiuIiEixyha0zCwO3AAcCwwBTjOzIS2SnQN84u77AdcCvwzXHQKcCgwFJgI3mlm8jTx/CVzr7oOBT8K8C5YBNAA/Bv53pzZcREQ6rJw9rTHAKnd/y90bgbnApBZpJgF3htP3AUdZMNTsJGCuu29297eBVWF+efMM1zkyzIMwzxNaK8PdN7r7MwTBS0REuoFyBq09gXdy5teEy/KmcfcksA7o18q6hZb3Az4N82hZVqEyimZm55pZnZnVffjhh+1ZVURE2qGcQcvyLPMi03TW8mLr0Sp3n+3ute5e279///asKiIi7VDOoLUG2CtnfgDwbqE0ZpYA+gAft7JuoeUfAX3DPFqWVagMERHpZsoZtBYDg8O7+ioJbqyY1yLNPOCscPpk4El393D5qeGdf4OAwcCiQnmG6zwV5kGY5wNtlCEiIt1Mou0kpeHuSTP7DvAYEAdud/dXzeynQJ27zwNuA+4ys1UEvZ9Tw3VfNbN7geVAErjQ3VMA+fIMi7wMmGtmPwNeDPOmUBlhXquBnYFKMzsBGO/uy0uzRUREpC2mTkXnqq2t9bq6unJXQ0QkUsxsibvXtpVOT8QQEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIUNASEZHIKGvQMrOJZrbCzFaZ2fQ87/cws3vC9583s4E5780Il68wswlt5Wlmg8I8VoZ5Vna0DBERKY+yBS0ziwM3AMcCQ4DTzGxIi2TnAJ+4+37AtcAvw3WHAKcCQ4GJwI1mFm8jz18C17r7YOCTMO92l9G5W0FERNqjnD2tMcAqd3/L3RuBucCkFmkmAXeG0/cBR5mZhcvnuvtmd38bWBXmlzfPcJ0jwzwI8zyhg2WIiEiZlDNo7Qm8kzO/JlyWN427J4F1QL9W1i20vB/waZhHy7LaW8ZWzOxcM6szs7oPP/yw1UaLiEjHlTNoWZ5lXmSazlrekTK2Xug+291r3b22f//++ZKIiEgnKGfQWgPslTM/AHi3UBozSwB9gI9bWbfQ8o+AvmEeLctqbxkiIlIm5Qxai4HB4V19lQQ3PcxrkWYecFY4fTLwpLt7uPzU8M6/QcBgYFGhPMN1ngrzIMzzgQ6WISIiZZJoO0lpuHvSzL4DPAbEgdvd/VUz+ylQ5+7zgNuAu8xsFUHv59Rw3VfN7F5gOZAELnT3FEC+PMMiLwPmmtnPgBfDvOlIGSIiUh4WdCqks9TW1npdXV25qyEiEilmtsTda9tKpydiiIhIZChoiYhIZChoiYhIZJTtRgwRkdY0NTWxZs0aGhoayl0V6URVVVUMGDCAioqKDq2voCUi3dKaNWvYaaedGDhwIMGT1STq3J21a9eyZs0aBg0a1KE8dHlQRLqlhoYG+vXrp4C1HTEz+vXrt029ZwUtEem2FLC2P9u6TxW0REQK6N27d3b64YcfZvDgwfzzn//k5ptvZs6cOR3Kc/Xq1fzhD3/IztfV1fHd7353m+u6o9BvWiIibXjiiSe46KKLePzxx9l7770577zzOpxXJmidfvrpANTW1lJb2+b/1JZUKpUiHo/GcIHqaYmItGLhwoVMnTqVhx56iH333ReAn/zkJ8yaNQuAcePGcdlllzFmzBj2339/Fi5cCASB4NJLL+Xggw9mxIgR3HLLLQBMnz6dhQsXUlNTw7XXXsvTTz/NN77xDQA2bNjA5MmTGT58OCNGjOD+++/fqj5Llizh8MMPZ/To0UyYMIH33nuvQ/V4+umnOeKIIzj99NMZPnw4AFdeeSUHHHAAxxxzDKeddhqzZs3izTffZNSoUdnyV65cyejRozt9OxdLPS0RkQI2b97MpEmTePrppznggAMKpksmkyxatIiHH36YK664ggULFnDbbbfRp08fFi9ezObNmxk7dizjx4/nqquuYtasWTz44INAEDwyrrzySvr06cOyZcsA+OSTT5qV09TUxEUXXcQDDzxA//79ueeee7j88su5/fbb210PgEWLFvHKK68waNAg6urquP/++3nxxRdJJpOMGjWK0aNHs++++9KnTx+WLl1KTU0Nd9xxB2effXYnbuX2UdASkW7vir+9yvJ313dqnkO+sDMzjxvaapqKigq++tWvctttt/Gb3/ymYLoTTzwRgNGjR7N69WoAHn/8cV5++WXuuy8YMH3dunWsXLmSysrKgvksWLCAuXPnZud32WWXZu+vWLGCV155hWOOOQYIelF77LFHh+sxZsyY7K3nzzzzDJMmTaK6uhqA4447LpvvlClTuOOOO7jmmmu45557WLSofANeFBW0zOx+4HbgEXdPl7ZKIiLdQywW49577+Xoo4/m5z//OT/84Q/zpuvRowcA8XicZDIYIN3duf7665kwYUKztLk9q5bcvdW769ydoUOH8uyzz3ZKPXr16tUs70JOOukkrrjiCo488khGjx5Nv379CqYttWJ7WjcBk4HrzOxPwO/c/fXSVUtEZIu2ekSl1LNnTx588EEOPfRQdt99d84555yi1pswYQI33XQTRx55JBUVFbzxxhvsueee7LTTTnz22Wd51xk/fjy//e1v+fWvfw0Elwdze1tf+tKX+PDDD3n22Wc55JBDaGpq4o033mDo0MLbp1A9Wvra177GtGnTmDFjBslkkoceeoipU6cCwVMsJkyYwPnnn89tt9221bpdqagbMdx9gbufAYwCVgPzzewfZjbZzDr2LA4RkYjYddddefTRR/nZz37GAw880PYKBJfUhgwZwqhRoxg2bBjTpk0jmUwyYsQIEokEI0eO5Nprr222zo9+9CM++eQThg0bxsiRI3nqqaeavV9ZWcl9993HZZddxsiRI6mpqeEf//hHh+rR0sEHH8zxxx/PyJEjOfHEE6mtraVPnz7Z98844wzMLPt7WLkUPZ6WmfUD/h34D4Jh5+8GvgYMd/dxpapg1Gg8LZHO8dprr3HggQeWuxo7lA0bNtC7d282bdrEYYcdxuzZs7N3Ds6aNYt169Zx5ZVXbnM5+fZtseNpFfub1p+BA4C7gOPc/b3wrXvMTEdoEZHtwLnnnsvy5ctpaGjgrLPOygasb37zm7z55ps8+eSTZa5h8b9p/dbd89a2mMgoIiLdX+6TOnL95S9/6eKaFFZs0OprZie2WLYOWObuH3RynURERPIqNmidAxwCZH4VHAc8B+xvZj9197tKUDcREZFmig1aaeBAd38fwMx2J7gN/svA3wl+6xIRESmpYp89ODATsEIfAPu7+8dAU+dXS0REZGvFBq2FZvagmZ1lZmcBDwB/N7NewKelq56ISPnE43FqamoYNmwYp5xyCps2bSqY9tNPP+XGG2/cpvJ+97vf8e6772bnp0yZwvLly1tdZ9y4cexI/2ZTbNC6ELgDqAEOAuYAF7r7Rnc/olSVExEpp+rqapYuXcorr7xCZWUlN998c8G0pQhat956K0OGDNmmPDtTKpUqdxXaDlpmFgfmu/v97n6xu3/f3e/zYv8rWURkO3DooYeyatUqAK655hqGDRvGsGHDso9cmj59Om+++SY1NTVceumlAFx99dXZIUFmzpwJBONpHXjggUydOpWhQ4cyfvx46uvrue+++6irq+OMM86gpqaG+vr6Zr2o888/n9raWoYOHZrNqzXb7RAm7t7mC5gH9Ckm7Y7+Gj16tIvItlu+fHm5q+C9evVyd/empiY//vjj/cYbb/S6ujofNmyYb9iwwT/77DMfMmSIv/DCC/7222/70KFDs+s+9thjPnXqVE+n055KpfzrX/+6/9d//Ze//fbbHo/H/cUXX3R391NOOcXvuusud3c//PDDffHixdk8cufXrl3r7u7JZNIPP/xwf+mll/Ku4+7e2NjohxxyiH/wwQfu7j537lyfPHlyNv0ll1zi7u4PPfSQH3XUUe7ufsstt/iVV17p7u4NDQ0+evRof+utt/ypp57ynj17+ltvveXu7osXL/aRI0f6pk2bfP369b7ffvv51Vdf7e7u48aNy7ZrxowZft111+Xdrvn2LVDnRRxji717sAFYZmbzgY05AU9jRItI6T0yHf7fss7N8/PD4dirWk1SX19PTU0NEPS0zjnnHG666Sa++c1vZp+QfuKJJ7Jw4UKOP/74Zus+/vjjPP744xx00EFA8IiklStXsvfeezNo0KBsvrnDiLTm3nvvZfbs2SSTSd577z2WL1/OiBEj8qbdnocwKTZoPRS+RER2GJnftHJ5kb+MuDszZsxg2rRpzZavXr06O4QIBDd71NfXt5rX22+/zaxZs1i8eDG77LILZ599Ng0NDa2Wvb0OYVJU0HL3O82sGtjb3Vd0ei1ERFrTRo+oKx122GGcffbZTJ8+HXfnL3/5C3fddddWQ45MmDCBH//4x5xxxhn07t2bf/3rX1RUtD4oRqFhS9avX0+vXr3o06cP77//Po888gjjxo0rmM/2PIRJsQ/MPQ6YBVQCg8ysBvipux/f+poiItuXUaNGcfbZZzNmzBgguCSWuQQ4duxYhg0bxrHHHsvVV1/Na6+9xiGHHAJA7969+f3vf088Hi+Y99lnn815551HdXV1s17SyJEjOeiggxg6dCj77LMPY8eObbWOmSFMvvvd77Ju3TqSySTf//73Ww1aU6ZMYfXq1YwaNQp3p3///vz1r3/dKl3uECZf/OIX8w5h8uc//7lkQ5gUNTSJmS0BjgSedveDwmXL3H14SWoVYRqaRKRzaGiS7mtbhzAp+dAkQNLd17UYBlq3vIuI7IDKOYRJsUHrFTM7HYib2WDgu0Drw2WKiMh2qZxDmBT7RIyLgKHAZuCPwHrg+6WqlIiISD7F3j24Cbg8fImIiJRFUT0tM9vfzGab2eNm9mTm1dFCzWxXM5tvZivDv7sUSHdWmGZl+KDezPLRZrbMzFaZ2XUW/thWKF8LXBemf9nMRhVRxn+a2TtmtqGj7RQRkc5V7OXBPwEvAj8CLs15ddR04Al3Hww8Ec43Y2a7AjMJxuwaA8zMCW43AecCg8PXxDbyPTYn7bnh+m2V8bdwmYiIdBPFBq2ku9/k7ovcfUnmtQ3lTgLuDKfvBE7Ik2YCwYN6P3b3T4D5wEQz2wPY2d2fDZ9XNSdn/UL5TgLmhI+4eg7oG+aTtwwAd3/O3d/bhjaKSIStXbuWmpoaampq+PznP8+ee+6ZnW9sbNwq/ccff9zqU+Azkskkffv2LUWVdwjFBq2/mdkFZrZHeAlu17CX0lG7ZwJC+PdzedLsCbyTM78mXLZnON1yeWv5tpZXvuXtYmbnmlmdmdV9+OGH7V1dRLqhfv36sXTpUpYuXcp5553HxRdfnJ2vrKzcKn2xQaurZR7TtL0oNmidRXA58B/AkvDV6n/QmtkCM3slz2tSkWVanmXeyvKuymvrFdxnu3utu9f279+/vauLSMT86le/yg5Ncv311wPB0CQrVqygpqaG6dOns379eo488khGjRrFiBEjePDBB9vM984772TMmDHU1NRwwQUXkE6nsz2z6dOnM3LkSA455BA++OADAN5//31OPPFEamtrGTNmDM899xwAP/rRj5g2bRrHHHMMkydPZuPGjZx00kmMHDmS0047jdraWpYuXcott9ySHUYF4KabbuIHP/hBCbZYJyrmUfCd/QJWAHuE03sAK/KkOQ24JWf+lnDZHsDr+dIVyjezbsvyC5XRoh4b2tM2DU0i0jm6w9AkGTNnzswOv/H888/7iBEjfOPGjb5+/Xo/4IAD/KWXXvKVK1f6yJEjs+s0Njb6+vXr3d39/fff9/3228/dg2FO+vTps1UZy5Yt80mTJnlTU5O7u0+dOtXvvvtub2pqcsAffvhhd3e/+OKL/Re/+IW7u3/rW9/yZ5991t292dAol19+uR988MFeX1/v7u6/+MUv/IILLnB396VLl3osFvMXX3zR169f7/vss0+2zIMPPthfffXVTtxy+ZVsaBIz+4G7/yqcPsXd/5Tz3s/d/YcdjJXzCHpvV4V/H8iT5jHg5zk3RowHZrj7x2b2mZl9BXgeOBO4vo185wHfMbO5BDddrHP398wsbxkdbJOIlMgvF/2S1z9+vVPzPGDXA7hszGXtXm/hwoWcdNJJ9OzZE4ATTjiBZ555Zqtn7bk7l112Gc888wyxWIx33nmHjz76qODvWQsWLGDx4sXU1gZPMqqvr2evvfYCgqfNH3vssUAwnEhm4MYFCxawYsWWZ5h/8skn2SfGT5o0iaqqKiAYTuSyy4K2jhw5MvsMwp122onDDjuMRx55hH322Yd4PN6tRkrOp63/0zoV+FU4PYPgLsKMiUBHg9ZVwL1mdg7wT+AUADOrBc5z9ylhcLoSWByu81N3/zicPh/4HVANPBK+CuYLPAz8G7AK2ARMBmitDDP7FXA60NPM1gC3uvtPOtheEdlOeJFDk8yZM4d169bxwgsvkEgkGDBgQJvDiXz729/e6pl9yWSy2W9oLYcTWbRoUd7f2IodTmTKlClcc801DBw4kMmTJxfVtnJqK2hZgel880Vz97XAUXmW1wFTcuZvB24vkG5YO/J14MICdSlUxg+Abn5xV2TH0JEeUakcdthhTJs2jUsvvZRUKsUDDzzAPffcs9WwIuvWreNzn/sciUSC+fPn869//avVfI8++mhOPvlkvve977Hbbruxdu1aNm7cyBe+8IVW17nhhhu4+OKLAVi6dGl2cMlcX/va17j33ns59NBDWbZsGcuXL8++N3bsWC688EIWL17MsmWdPNBmCbQVtLzAdL55EZHt3pgxYzjttNM4+OCDATj//PMZPjwY8KK2tpbhw4fz9a9/nUsuuYTjjjuO2tpaRo0axeDBg1vNd/jw4cycOZOjjz6adDpNRUUFN998c6tB64YbbuD888/njjvuIJlMcsQRR3DDDTdsle6iiy7izDPPZMSIEYwaNYphw4Y1G07k5JNP5vXXX2+2rLtqdWgSM0sBGwl6VdUEl9YI56vcvfURzXZAGppEpHNoaJLOk0wmSSaTVFVVsXLlSsaPH8/KlStJJIJ+y8SJE5kxYwaHH354l9SnZEOTuHvh0cpERCQSNmzYwFFHHUUymcTdueWWW0gkEqxdu5ZDDjmE0aNHd1nA2lbFDk0iIiIR1bdvX5Ys2fohRv369eONN94oQ406rth/LhYRESk7BS0R6baKvb1comNb96mCloh0S1VVVaxdu1aBazvi7qxduzb7T88dod+0RKRbGjBgAGvWrEEPod6+VFVVMWDAgA6vr6AlIt1SRUUFgwYNKnc1pJvR5UEREYkMBS0REYkMBS0REYkMBS0REYkMBS0REYkMBS0REYkM3fLeTXz7sW+zet3qcldD2sE6PqTcNvEyjwrUFe0uto2l2hbFtrHU22Jb29fVn5VHT3qUHvEeJS1DQaubGL37aPbeae9yV0O6iONlC3rlVIp2m3Vufp39BI5y7+vO3j6tiVnpL94paHUTF9bkHVhZRERy6DctERGJDAUtERGJDAUtERGJDAUtERGJDAUtERGJDAUtERGJDAUtERGJDAUtERGJDAUtERGJDAUtERGJDAUtERGJDAUtERGJDAUtERGJDAUtERGJDAUtERGJDAUtERGJjLIELTPb1czmm9nK8O8uBdKdFaZZaWZn5SwfbWbLzGyVmV1n4dCchfK1wHVh+pfNbFRrZZhZTzN7yMxeN7NXzeyq0m4REREpRrl6WtOBJ9x9MPBEON+Mme0KzAS+DIwBZuYEt5uAc4HB4WtiG/kem5P23HD9tsqY5e4HAAcBY83s2M5puoiIdFS5gtYk4M5w+k7ghDxpJgDz3f1jd/8EmA9MNLM9gJ3d/Vl3d2BOzvqF8p0EzPHAc0DfMJ+8Zbj7Jnd/CsDdG4EXgAGd1noREemQcgWt3d39PYDw7+fypNkTeCdnfk24bM9wuuXy1vJtLa98y7PMrC9wHEHPTUREyihRqozNbAHw+TxvXV5sFnmWeSvLOz0vM0sAfwSuc/e3CmZudi7BZUf23nvvNqoiIiIdVbKg5e5HF3rPzN43sz3c/b3wMt0HeZKtAcblzA8Ang6XD2ix/N1wulC+a4C98qxTqIyM2cBKd/91obYAuPvsMC21tbVtBVAREemgcl0enAdk7gY8C3ggT5rHgPFmtkt4c8R44LHwst9nZvaV8K7BM3PWL5TvPODM8C7CrwDrwnzylgFgZj8D+gDf77RWi4jINilX0LoKOMbMVgLHhPOYWa2Z3Qrg7h8DVwKLw9dPw2UA5wO3AquAN4FHWssXeBh4K0z/f4ELWivDzAYQXMYcArxgZkvNbEopNoSIiBTPghvwpLPU1tZ6XV1duashIhIpZrbE3WvbSqcnYoiISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGQoaImISGSUJWiZ2a5mNt/MVoZ/dymQ7qwwzUozOytn+WgzW2Zmq8zsOjOz1vK1wHVh+pfNbFQRZTxqZi+Z2atmdrOTqA9kAAALXUlEQVSZxUu3RUREpBjl6mlNB55w98HAE+F8M2a2KzAT+DIwBpiZE9xuAs4FBoeviW3ke2xO2nPD9dsq41vuPhIYBvQHTumUlouISIclylTuJGBcOH0n8DRwWYs0E4D57v4xgJnNByaa2dPAzu7+bLh8DnAC8Egr+U4C5ri7A8+ZWV8z2yNMu1UZwB/dfX2YTwKoBLxTWt4R6TSkGqFpEyQboKk+mG6qz3mF88n6FssLvbcJmhogtRksDrFE+IqHrwqoqIbKXsGromfzv8VOxys61mZ3SCeb1zfZkKfdRbyXb7tAizaH04keOW3oCRVhO7LTPaGy95b3801XVEPQ+e/Avk5B48ac12c50xtg84Zgumlj0I7GTWEbNzWfTjaAp4PPjqfBU0HeEOyTeAXEK8NXznSiaksbmr16Nv/bLF3u3ypIVEOsHefD6XSwfzLtbNrUYnpT0PZsGzeGyzZumc60O9UUtjsVtDkzHUts3dbMdKIyqHNuexPVQVsqeoZtrc5pd6E0PSHejkNqKhnUP7NPGzc039fZ+dx9vHHLZzoznWoM93OqeduxnHZWbN3+7D7LfNZ7btmXlb2av99sWa8t26Cjn/NtUK6gtbu7vwfg7u+Z2efypNkTeCdnfk24bM9wuuXy1vJtLa98ywEws8cIemCPAPe1p4Htdve3YO3K4EuX3Bx8EFNNQVBJJzuWZ6wi54CSc5BJVEPP3YLpeGX4QU8GH/Z0Mnw1BV+aDe83P5A0bWp/HSp7BV/qWBwsFrwy01jQxlRT0OZkY9j2Rjp2nmBbH0Qz05W9odfngmVYTltTW9qc3Azr321+wGza2M59YFsOBLFEcFJgseBAbuFJQTq5pc2ppuCVbgqCTdHFxLc+uGQOPlV9wrJjzbc5bCkvs52TjcGBM9W0dXBP1rdn42+RyNnuiarg4NZsW4d/Mycb7RGryH9CUdU3OChbvPm2tlhQXnZbh38bN0Lqky0ngsmG4ESuaVOwLzoilsgJgFUQ7xHklQo/X822/eb25Zs5acoEjsrw5DC+65Z2Zr9jccC33teZdic355zUhUEw1dj+9la0CHbnPh3Uq4RKFrTMbAHw+TxvXV5sFnmWeSvLOz0vd59gZlXA3cCRwPy8mZudS3DZkb333ruNqhSw22Co2jnnLDA8K0r0yDkzynMGnP2CZA7QOYGpPWd9xcqeFbc4+y003RSeOTY78885I8SDL3bLtmbbnHPw26q9eXoD8crSnP0lG/O3q9CZfybopZNBrzHT08m0PZYIDr65Z8CxxJYeamWvIMhW9s6Z7gU9Mj27XqVra650Ok/vPl/PNue9pjzvQZ6ebTwIaIV66vl6sxW9gp5RqaVTzQN3pk3ZbZFveZ40qcawl1fRvLcXSzTfp9n9nHnt1Dwgd0WbU03NA1njpubzbS7bGHyHS6xkQcvdjy70npm9b2Z7hL2hPYAP8iRbw5ZLfQADCC73rQmnc5e/G04XyncNsFeedQqVkduOBjObR3CJMW/QcvfZwGyA2trajl1GnPCfHVqty8ViW75Y9C93bbpOohISuwK7lrsmXSsWCwNHT6BfuWvTdWLxIJj06F3umnSdzGXEqp3LXZNWletGjHlA5k69s4AH8qR5DBhvZruEN0eMBx4LL/99ZmZfCe8aPDNn/UL5zgPODO8i/AqwLswnbxlm1jsMephZAvg34PVOa72IiHRIuX7Tugq418zOAf5JeGeemdUC57n7FHf/2MyuBBaH6/w0c8MEcD7wO6Ca4PemR1rLF3iYIPCsAjYBkwEKlWFmuwPzzKwHEAeeBG7u5G0gIiLtZMENddJZamtrva6urtzVEBGJFDNb4u61baXTEzFERCQyFLRERCQyFLRERCQyFLRERCQyFLRERCQydPdgJzOzD4H/6eDquwEfdWJ1okBt3jGozTuOjrb7i+7e5hMLFLS6ETOrK+aWz+2J2rxjUJt3HKVuty4PiohIZChoiYhIZChodS+zy12BMlCbdwxq846jpO3Wb1oiIhIZ6mmJiEhkKGh1A2Y20cxWmNkqM5te7vp0BTO73cw+MLNXyl2XrmJme5nZU2b2mpm9ambfK3edSs3MqsxskZm9FLb5inLXqauYWdzMXjSzB8tdl65gZqvNbJmZLTWzkj01XJcHy8zM4sAbwDEEg1IuBk5z9+VlrViJmdlhwAZgjrsPK3d9ukI4Rtse7v6Cme0ELAFO2J73dTjmXS9332BmFcAzwPfc/bkyV63kzOwSoBbY2d2/Ue76lJqZrQZq3b2k/5umnlb5jQFWuftb7t4IzCUYJXm75u5/Bz5uM+F2xN3fc/cXwunPgNeAPctbq9LywIZwtiJ8bfdnymY2APg6cGu567K9UdAqvz2Bd3Lm17CdH8gEzGwgcBDwfHlrUnrhZbKlwAfAfHff7tsM/Br4AZAud0W6kAOPm9kSMzu3VIUoaJWf5Vm23Z+J7sjMrDdwP/B9d19f7vqUmrun3L0GGACMMbPt+nKwmX0D+MDdl5S7Ll1srLuPAo4FLgx/Auh0ClrltwbYK2d+APBumeoiJRb+rnM/cLe7/7nc9elK7v4p8DQwscxVKbWxwPHhbzxzgSPN7PflrVLpufu74d8PgL8Q/PTR6RS0ym8xMNjMBplZJXAqMK/MdZISCG9KuA14zd2vKXd9uoKZ9TezvuF0NXA08Hp5a1Va7j7D3Qe4+0CC7/OT7v7vZa5WSZlZr/DmIsysFzAeKMmdwQpaZebuSeA7wGMEP8zf6+6vlrdWpWdmfwSeBb5kZmvM7Jxy16kLjAX+g+DMe2n4+rdyV6rE9gCeMrOXCU7Q5rv7DnEL+A5md+AZM3sJWAQ85O6PlqIg3fIuIiKRoZ6WiIhEhoKWiIhEhoKWiIhEhoKWiIhEhoKWiIhEhoKWSBczs75mdkHO/BfM7L4SlXWCmf2fUuTdRrmVZvZ3M0t0ddmyfdMt7yJdLHzu4INd8XR7M/sHcHypnrxtZonwfw3zvTeT4GHQd5eibNkxqacl0vWuAvYN/7n4ajMbmBlXzMzONrO/mtnfzOxtM/uOmV0Sjsv0nJntGqbb18weDR9OutDMDmhZiJntD2x294/MbKcwv4rwvZ3D8Y8qCuVlZseZ2fNh2QvMbPdw+U/MbLaZPQ7MMbOh4ZhZS83sZTMbHFbhr8AZJd+askNR0BLpetOBN929xt0vzfP+MOB0gme3/Sewyd0PIniCyJlhmtnARe4+GvjfwI158hkL5A6F8jTBcBkQPF7ofndvaiWvZ4CvhGXPJXhqecZoYJK7nw6cB/wmfChuLcHzNCF4jM/BRW0RkSLperNI9/NUGGQ+M7N1wN/C5cuAEeFT4r8K/Cl4nCEAPfLkswfwYc78rQSB56/AZGBqG3kNAO4JB6+sBN7OyWueu9eH088Cl4djSP3Z3VdC8HR3M2s0s53C9ohsM/W0RLqfzTnT6Zz5NMGJZgz4NOypZV4H5smnHqjKzLj7fwMDzexwIO7ur7SR1/XAb919ODAtNy9gY06+fwCOD8t7zMyOzEnXA2ho7wYQKURBS6TrfQbs1NGVwzG43jazUyB4eryZjcyT9DVgvxbL5gB/BO4oIq8+wL/C6bMK1cfM9gHecvfrCEYoGBEu7wd8GF6CFOkUCloiXczd1wL/bWavmNnVHczmDOCc8KnarwKT8qT5O3CQ5Vz3A+4GdiEIXG3l9ROCy4YLgdbuPvxfwCvh6MQHEARGgCOAh9vTKJG26JZ3ke2Ymf0G+Ju7LwjnTya4geI/uqDsPwMz3H1FqcuSHYduxBDZvv0c+DKAmV1PMBR6ycfwCgc0/asClnQ29bRERCQy9JuWiIhEhoKWiIhEhoKWiIhEhoKWiIhEhoKWiIhEhoKWiIhExv8HeONXWOiL85AAAAAASUVORK5CYII=\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    },
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "time = np.linspace(0.0, tfinal, num=nsteps, endpoint=False)/(2.0*math.pi) # in years\n",
    "plot.plot(time, E_results[:,0], label='Kinetic energy')\n",
    "plot.plot(time, E_results[:,1], label='Potential energy')\n",
    "plot.plot(time, E_results[:,2], label='Total energy')\n",
    "plot.legend()\n",
    "plot.xlabel('time (years)')\n",
    "plot.ylabel('Energy')\n",
    "plot.show()\n",
    "plot.plot(time, (E_results[:,2]-E_results[0,2])/E_results[0,2], label='Total energy change')\n",
    "plot.xlabel('time (years)')\n",
    "plot.ylabel('Fractional Energy error')\n",
    "plot.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Pretty small error!  Let's see how the 4th order RK integrator does."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 17,
   "metadata": {},
   "outputs": [],
   "source": [
    "def rk4step (derivatives, start, delta, values, m):\n",
    "    \"\"\"Takes a single Runge-Kutta 4th order step for any number of \n",
    "    derivatives dy/dx = derivatives(x, y), where the derivatives function\n",
    "    takes a position start and an array of values (y) and returns an equivalent\n",
    "    array of derivatives, starting at x_j = start, with a step of size delta = x_j+1 - x_j  \n",
    "    and previous values y[i](x_j) = values[i] (a numpy array).\n",
    "    returns the array of new values for the next step.\"\"\"\n",
    " \n",
    "    k1 = delta * derivatives (start,             values         , m)    \n",
    "    k2 = delta * derivatives (start + delta/2.0, values + k1/2.0, m)\n",
    "    k3 = delta * derivatives (start + delta/2.0, values + k2/2.0, m)\n",
    "    k4 = delta * derivatives (start + delta,     values + k3    , m)\n",
    "\n",
    "    #return new values y_j+1.  Note these are all equal length arrays.\n",
    "    return values + (1.0/6.0)*(k1 + 2.0*(k2 + k3) + k4)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "define derivatives (remember this is a multi-dimension problem)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "metadata": {},
   "outputs": [],
   "source": [
    "def dxdt_dvdt(t, xv, m):\n",
    "    \"\"\"returns derivatives dx/dt and dv/dt\n",
    "       xv is an array which contains x's and v's for all particles\"\"\"\n",
    "\n",
    "    deriv = np.zeros_like(xv) # set up derivatives array, same size as xv\n",
    "    deriv[:, 0:2] = xv[:, 2:4]   #derivatives of x are v's for all planets\n",
    "    deriv[:, 2:4] = ComputeAccel(xv[:,0:2], m)  #derivatives of v are accelerations\n",
    "    return deriv"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Now do the integration"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 19,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAYYAAAD8CAYAAABzTgP2AAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADl0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uIDIuMi4zLCBodHRwOi8vbWF0cGxvdGxpYi5vcmcvIxREBQAAIABJREFUeJzt3XmcFPWd//HXBxA8YhBkEEQR9EcieESSCcZ4JDEeaBIx0Rg1q+iaKImuOVWMCSAeQZP8NK5GxSskZkXirko847muWTEOCYiACgEPBGUUIRoPBD77R9Wkqme6Z3qme/rbXf1+Ph796G9Vfbv7XTMwn66qb1WZuyMiItKiR+gAIiJSXVQYREQkhwqDiIjkUGEQEZEcKgwiIpJDhUFERHKoMIiISA4VBhERyaHCICIiOXqFDtAVAwYM8GHDhoWOISJSU+bOnfu6uzd01K8mC8OwYcNoamoKHUNEpKaY2YvF9NOuJBERyaHCICIiOVQYREQkhwqDiIjkKEthMLMbzWy1mT1TYLmZ2RVmttTMnjazj6eWjTezJfFjfDnyiIhI15Vri+HXwNh2lh8GjIgfpwJXA5hZf2AysDcwBphsZv3KlElERLqgLIXB3R8D1rTTZRzwG4/MAbYxs8HAocAD7r7G3d8EHqD9AiMiIt2sUucxDAFeTk2viOcVmi9SPdxh0Z1w+2mw4b3S3+8TJ8FB58MW25T+XiLdoFKFwfLM83bmt30Ds1OJdkMxdOjQ8iUTAVj+GPzXafDWyu7/rLm/jh6F7P0tOPDH0OdD3Z9FJI9KFYYVwI6p6R2AlfH8z7aa/2i+N3D36cB0gMbGxrzFQ6QomzbBjYfCij93/rU9e8ORV8NuXwbrxJ7YtS/BvWfD8/d13PfJq6NH2oTHYdAencsq0kWVKgyzgTPMbCbRgeZ17r7KzO4HLk4dcD4EOLdCmaSe3P4tmP8fxfX94mXw8ZOgRxlHc/fbCY6/tfDy15fAXd+DF/4n//Jr9sud/s7T0XuKdIOyFAYzu4Xom/8AM1tBNNJoMwB3vwa4BzgcWAq8A5wcL1tjZhcAT8VvNdXd2zuILVKcRy+BRy9uv89HvwDH/g4s3x7NChswAk66q+38F5+Am/KMx/jlnrnTZy+HLft3TzapO+Zee3tlGhsbXRfRkzb+90r443mFlzeMjHbJ9KzJa0dGFs2GWSe032fSGujRszJ5pKaY2Vx3b+yoXw3/DxEB1r8DFw/Ov6xnHzhnOfTeqrKZutOoI2DKumT6z9fBPT/M7TM13nL44mXQ+K+VyyaZoS0GqU0PXwiP/azt/B33hlP+WPk81WDlPJj+mfzLfrwaevWpbB6pOtpikOx5bx1MKzBU+Tvzod+wisapOtvvlWxNXPUpaF6cLLtwYPT8+Umw/w8qn01qirYYpPo9fBE8dmnb+R89HI67pfJ5asnrS+DKAl8QJ71Z3pFXUvW0xSC1b+mDcPNRbef/4DnYelDl89SiASOSrYjfjINljybLpsajxNPHLERQYZBq9Nar8IuP5s4bNQ6O+U2YPFlx4p3R87pX4LJRyfwpfbX1JTm0K0mqx6aNyYiaFtYDJr8ZJk/WvTQnOgM87Yh/h4+fGCaPdLtidyWpMEh1mNK37bzJa6vj5LOse+zn8PAFufO+PQcGjgyTR7qNCoPUht99FZa0Gl56zou68mgI1+wHry7Infejldk6D6TO6eCzVLd5t8AdE3LnfeMh2KHDf7PSXSY8Hj2nt94u3j6epwPU9USFQSqv9W6jg86H/b4bJou0NWUdbFgPFzak5vWFM5qiUU6SeSoMUjlvr4afp/6wbNEPznkhWBxpR6/eUYFYswyuGB3Nu7IRBu0JEwpcAVYyQ2e3SGXcdkpuUTj1URWFWtB/59zdSK8+HW091OCxSSmeCoN0vyl94ZnbUtPrYPvR4fJI501ZF91VrsX520RXepVMUmGQ7vPyU7nHE0b/iw5i1rIDzoLzXk2mZ52Qf5ix1DwdY5DucfEQWP92Mn3WMthq23B5pDw22yIq7umCMKUvTHwJNleRyIqybDGY2Vgze87MlprZxDzLLzOzefHjeTNbm1q2MbVM26a1buMH0R+KdFGYsk5FIWumrIMT7kimpw2F+34ULo+UVcknuJlZT+B54GBgBdFtOo9z90UF+v8bMNrd/zWeftvdP9SZz9QJblXqzRfglx9Lpsf9CkZ/PVgcqQD36HhDmnYXVq1iT3ArxxbDGGCpuy9z9/XATGBcO/2PA3S1rqx5ZW5uUZi0RkWhHphFhWDnzyXzdNyh5pWjMAwBXk5Nr4jntWFmOwHDgYdTszc3syYzm2NmR5Yhj1Tas3fDdQcm01PW6Z7D9ebEO+BzqVFLKg41rRyFId9VzgrtnzoWuM3dN6bmDY03bY4HLjezXfJ+iNmpcQFpam5uLi2xlM+T18LM45Np7UaoX585C75yXTKt4lCzylEYVgA7pqZ3AFYW6HssrXYjufvK+HkZ8CiQd4C7u09390Z3b2xoaMjXRSrt3olw79nJtIqC7HkMnHR3Mq3iUJPKURieAkaY2XAz6030x7/N6CIz+yjQD3giNa+fmfWJ2wOAfYG8B62lyvz2K/Dk1cm0ioK0GLYfnP5UMq0zpWtOyYXB3TcAZwD3A4uBWe6+0MymmtkRqa7HATM9dxjUSKDJzOYDjwDTCo1mkiryi13hbw9F7c22VFGQtho+Aj9ckkyfv010IyapCbofg3ROetfADp+EbzwYLotUv/XvwMWDk2nd3yGoSg5XlXrR+vIWKgrSkd5bRkOXW1y8Pby7tnB/qQoqDFKcq/ZO2p+fDOOuCpdFakuPnrm7Gy/ZKVwWKYoKg3Tsr7+D5mej9qfPhP2/HzaP1KZ0cdBopaqmwiDte+tVuPPbyfQhFxTuK9KRs5cnbRWHqqXCIIW5wy8+mkxr9JGUasv+cMxvkuk7Tw+XRQpSYZDC0hdHU1GQchk1DnbaL2r/9WZ48Yn2+0vFqTBIfunN/B8VOpFdpItOTp0dfdNY+ODdcFmkDRUGaevmo5P2+D9o3Ll0j/RW6EWDwuWQNlQYJNdz98HSB6L27kfD8APC5pFsm5w6p0EHo6uGCoMk3lsHt3wtmT76hnBZpD6YwfcXJ9MqDlVBhUES04YmbR1slkr58PbwpSuS6fkzw2URQIVBWlwwMGmrKEilfWI8bNEvat9+WtgsosIgwLtvwsb3o/bXbg6bRerXOS8kbe1SCkqFQeCSYUl75JeCxRDh23OS9msLw+WocyoM9e72byXtybrqpQQ2cGTSvvrT4XLUORWGeuYO8/8jao85LRohIhJa+hjXz0aEy1HHylIYzGysmT1nZkvNbGKe5SeZWbOZzYsf30gtG29mS+LH+HLkkSKlL3lx+KXhcoi09pXroud/rIb3/h42Sx0quTCYWU/gKuAwYBRwnJmNytP1VnffK35cH7+2PzAZ2BsYA0w2s36lZpIiPD0raZ+7IlwOkXz2PCZpT9sxXI46VY4thjHAUndf5u7rgZnAuCJfeyjwgLuvcfc3gQeAsWXIJB35r29Gzx8aBH22DptFJJ/0Ma+7fxAuRx0qR2EYArycml4Rz2vtKDN72sxuM7OWrwDFvlbKKT0U8IfPhcsh0h6z6BayAE9dHx0Tk4ooR2HId8Sy9W/wD8Awd98TeBCY0YnXRh3NTjWzJjNram5u7nLYutecKgTf0uWOpcqlbyGbPiYm3aochWEFkN4JuAOQc51md3/D3eMzqLgO+ESxr029x3R3b3T3xoaGhjLErlNXjUna2+U7FCRSZc55MWn/4/VwOepIOQrDU8AIMxtuZr2BY4HZ6Q5mNjg1eQTQctWs+4FDzKxffND5kHiedIe1LyVtXfZCasUWqS2Fn+0SLkcdKbkwuPsG4AyiP+iLgVnuvtDMpprZEXG3M81soZnNB84ETopfuwa4gKi4PAVMjedJd7h8j9AJRLrmx6ndxxvWh8tRJ8xr8IBOY2OjNzU1hY5RW95/G34aH9eftAZ69AybR6Sz0oMmtMXbJWY2190bO+qnM5/rxU9Tg71UFKQWnfW3pF2DX2hriQpDPdi0KWlPfLlwP5FqttWApK0RSt1KhaEeTE2dTL75h8PlECnVhD+FTlAXVBjqyZnzQicQKc2g3ZN2+o6DUlYqDFmXPmDXf3i4HCLlcmx8ReD3dAC6u6gw1IvxfwidQKQ8dv1C0r7l+HA5MkyFIctuOCRpDz8gXA6Rcvv8pOj5ubvD5sgoFYYse/nJ6Pnwn4fNIVJu+6eutrroznA5MkqFIavS+1/HfDNcDpHuNuvE0AkyR4UhqzRiQ7LuJ7qgXndRYci6Ux4MnUCke/TcLGkv/59wOTJIhSGLNryftHf8ZLgcIpUy44uhE2SKCkMWXTgwdAKRyvhR3tu3SIlUGLLs+FmhE4h0r95bJe1VT4fLkTEqDFmzaWPS/sih4XKIVNq1+4dOkBkqDFlz0aDQCUQq6+zloRNkTlkKg5mNNbPnzGypmU3Ms/z7ZrbIzJ42s4fMbKfUso1mNi9+zG79WumkjfHdrcb9KmwOkUrZsn/SXqMiUQ4lFwYz6wlcBRwGjAKOM7PWd5n/K9Do7nsCtwGXppa96+57xY8jkK5L37xk9NfD5RAJ5Yq9QifIhHJsMYwBlrr7MndfD8wExqU7uPsj7v5OPDkH2KEMnyutXamhqVKnvrcwdIJMKUdhGAKkbwu2Ip5XyCnAvanpzc2syczmmNmRhV5kZqfG/Zqam5sLdatvbyyJng/8SdgcIpXWN/VdU7f9LFk5CoPlmZf3N2Nm/wI0Aj9LzR4a35z6eOByM9sl32vdfbq7N7p7Y0NDQ6mZs+2AH4ZOIBLOo9NCJ6h55SgMK4AdU9M7AG3OOjGzg4DzgCPc/Z+n5rr7yvh5GfAoMLoMmUSkXv23CkOpylEYngJGmNlwM+sNHAvkjC4ys9HAtURFYXVqfj8z6xO3BwD7AovKkKn+vDQndAKRsE7U5bfLpVepb+DuG8zsDOB+oCdwo7svNLOpQJO7zybadfQh4PdmBvBSPAJpJHCtmW0iKlLT3F2FoStu1MlsUud2/mzoBJlRcmEAcPd7gHtazZuUah9U4HX/C+xRjgwS2+eM0AlEpMbpzOesOeTC0AlEwnvuvtAJapoKQ9ZYvkFiInXmlq+FTlDTVBiy4K3XQicQqQ7aYi4LFYYsmP6Z0AlEqsOn/y10gkxQYciCt1ZFz0M+ETaHiGSCCkOWaBy3SOL1paET1CwVhizps3XoBCLVQzfu6TIVBhHJpg/e6biP5KXCICLZsusXQyeoeSoMIpItww8InaDmqTDUurdXd9xHpJ4M2y90gpqnwlDrntep/yI5GkaGTlDzVBhq3bN3h04gUl166M9aqfQTrHXaYhCRMlNhEBGRHGUpDGY21syeM7OlZjYxz/I+ZnZrvPxJMxuWWnZuPP85M9PdZrqq79DQCUQkI0ouDGbWE7gKOAwYBRxnZqNadTsFeNPd/x9wGXBJ/NpRRLcC3Q0YC/wqfj/prF2/EDqBiGREObYYxgBL3X2Zu68HZgLjWvUZB8yI27cBn7foHp/jgJnu/r67LweWxu8nnaXCICJlUo7CMAR4OTW9Ip6Xt4+7bwDWAdsW+VopxtB9QicQkYwoR2HId8swL7JPMa+N3sDsVDNrMrOm5ubmTkasAz20B05EyqMchWEFsGNqegdgZaE+ZtYL6AusKfK1ALj7dHdvdPfGhoaGMsTOmNeeCZ1ARDKiHIXhKWCEmQ03s95EB5Nnt+ozGxgft48GHnZ3j+cfG49aGg6MAP5chkz1Z/FdoROISEaUXBjiYwZnAPcDi4FZ7r7QzKaa2RFxtxuAbc1sKfB9YGL82oXALGARcB9wurtvLDVTXXpWhUFEyqNXOd7E3e8B7mk1b1Kq/R7w1QKvvQi4qBw56pp2JYlImejMZxERyaHCUOt22jd0AhHJGBWGWqcT20SkzFQYap1uYyiS680XQyeoeSoMta7fTqETiFSXFx4PnaDmqTCISLa8+KfQCWqeCoOIZMvCO0InqHkqDCKSLR/8I3SCmqfCkCWbdNK4yD/96x9DJ6hZKgxZMvvM0AlEqsfQvUMnqFkqDFky7+bQCUQkA1QYsmCCRmGIALDoztAJMkGFIQsG7R46gUh1mHVi6ASZoMIgItlz2KWhE9Q0FYasaboxdAKR8PY+LXSCmqbCkDV3fS90AhGpcSUVBjPrb2YPmNmS+Llfnj57mdkTZrbQzJ42s6+llv3azJab2bz4sVcpeeraUTeETiASVvPzoRNkRqlbDBOBh9x9BPBQPN3aO8CJ7r4bMBa43My2SS0/y933ih/zSsxTv/Y4OnQCkbCu+mToBJlRamEYB8yI2zOAI1t3cPfn3X1J3F4JrAYaSvxcEZH8Ro0LnaDmlVoYtnP3VQDx88D2OpvZGKA38LfU7IviXUyXmVmfEvMIwPP3h04gEs5Xrg+doOZ1WBjM7EEzeybPo1Nl2cwGA78FTnb3TfHsc4FdgU8C/YFz2nn9qWbWZGZNzc3Nnfno+vMfx4ROIFJZ61MXzuvVO1yOjOiwMLj7Qe6+e57HncBr8R/8lj/8q/O9h5l9GLgb+LG7z0m99yqPvA/cBIxpJ8d0d29098aGBu2Jyuu810InEAnj4u1DJ8iUUnclzQbGx+3xQJvz0c2sN3A78Bt3/32rZS1FxYiOTzxTYp76ttnmSXtFU7gcIqGMvyt0gkwotTBMAw42syXAwfE0ZtZoZi07+o4BDgBOyjMs9XdmtgBYAAwALiwxj7S4/vOhE4hUxsYPkvbw/cPlyJBepbzY3d8A2vwFcvcm4Btx+2Yg72U/3f3AUj5f8pj4EkwbGjqFSOVcMCB0gszRmc9Zs3nfpK0TfqSefHVGx32kKCoMWaYTfiTr3JP2bm1Oo5IuUmHIoh9oS0HqxM9HhE6QSSoMWbT1dkl73Svhcoh0t3/E5zQd9rOwOTJGhSHrLhsVOoFI99v71NAJMkWFIavO/GvoBCLd6zoNye4uKgxZ1X/npL1ibrgcIt3llfgkzn2/GzZHBqkw1IPrdbqIZMzSB5P2weeHy5FRKgxZNmVd0v77ynA5RMrt5qOiZ9OfsO6gn2q9+P8jQycQKY/XlybtyW+Gy5FhKgxZ95M3kvb6d8LlECmXKz8ROkHmqTBkXc/U5bAuHhwuh0g5vLs2aU9eW7iflESFoR6c82LSTl9CQKTWXLJT0jYLlyPjVBjqwRbbJO3ztyncT6SabdqYtH+0KlyOOqDCUC/+7S+hE4iUZmr/pN17y3A56oAKQ73YdpekPaVv4X4i1e57i0InyLySCoOZ9TezB8xsSfzcr0C/jam7t81OzR9uZk/Gr781vg2odJcT7gidQKRr0l9m+g4Jl6NOlLrFMBF4yN1HAA/F0/m86+57xY8jUvMvAS6LX/8mcEqJeaQ9u3wuaf9C5zVIjUgPmDjlwcL9pGxKLQzjgJbbJs0Air5ThpkZcCBwW1deL1305Wuj57dWwvp/hM0iUoz0gIkddfOpSii1MGzn7qsA4ueBBfptbmZNZjbHzFr++G8LrHX3DfH0CkDbiN3tY8cm7Yu3D5dDpBgLbkvaE18Ol6POdFgYzOxBM3smz2NcJz5nqLs3AscDl5vZLkC+QcgFB9mb2alxcWlqbm7uxEdLG5NSlxG4/7xwOUQ68p/x3uWtGmDzD4fNUkc6LAzufpC7757ncSfwmpkNBoifVxd4j5Xx8zLgUWA08DqwjZm1nJq7A1DwSm/uPt3dG929saGhoROrKG306AF7HBO1n7hSJ71JdUofcD5raeF+Unal7kqaDYyP2+OBO1t3MLN+ZtYnbg8A9gUWubsDjwBHt/d66SZHXZe0ddKbVJvm1H3LJ/wpXI46VWphmAYcbGZLgIPjacys0cyuj/uMBJrMbD5RIZjm7i0Dkc8Bvm9mS4mOOdxQYh7pjLOXJ+1n7w6XQ6S1q1IHmQftHi5HnTKvwd0IjY2N3tTUFDpGNkzdFjbFx//T928QCeWK0bBmWdTWv8myMrO58fHedunM53o3KXVZbp0RLaGt/0dSFI64MmyWOqbCIPDNh5N2y39KkRDSQ6g/fkK4HHVOhUFgSOrGJ1eMDpdD6lt66PQk3ZktJBUGiaT35WqXklTa6mejodMAo46MhlRLMPrpS+Lk+5L2jYeFyyH1ZdNG+NXeyfQxMwr3lYpQYZDETvvA6Hi/7kv/C4t0WolUQPo+CxqFVBVUGCTXuNRIkFknwjtrwmWR7Evvtpykf2vVQoVB2kp/a7t0eLgckm2X75G0v/UE9OgZLovkUGGQ/HQwWrpT002w9qWofcDZsN2osHkkhwqDFDbxpaSt4iDlsu4VuOu7yfSBusJvtVFhkMI27wvH/z6ZnjW+cF+RYrjDZamtAx1srkoqDNK+jxwCHxkbtRfdAX97JGweqW3pK/mqKFQtFQbp2PG3Ju3fHgmvLSrcV6SQ9O7I814Nl0M6pMIgxUl/u7t6H205SOeki8LJ98FmW4TLIh1SYZDipYvDb4+Ev94cLovUjnRR+ObD0YmUUtVUGKRz0sXhztPh4QvDZZHq5p5bFL4zP/eCjVK1SioMZtbfzB4wsyXxc788fT5nZvNSj/fM7Mh42a/NbHlq2V6l5JEKSReHx34WnSEtkrbxg9wDzee8AP2GhUojnVTqFsNE4CF3HwE8FE/ncPdH3H0vd98LOBB4B/hjqstZLcvdfV6JeaRS0sVh0Z1w5ScL95X68t7f4YIByfSPV8MWbb4zShUrtTCMA1ouhTgDOLKD/kcD97r7OyV+rlSDKeugYWTUfv15nQQn0clr03ZMpievhV59wuWRLim1MGzn7qsA4ueBHfQ/Fril1byLzOxpM7vMzAr+CzKzU82sycyampubS0st5XP6HNjtK8m0ikP9enVB25PXzMLlkS7rsDCY2YNm9kyex7jOfJCZDQb2AO5PzT4X2BX4JNAfOKfQ6919urs3untjQ0NDZz5auttXb4LPpH51Kg71Z/FdcM1+ybROXqtpvTrq4O4HFVpmZq+Z2WB3XxX/4V/dzlsdA9zu7h+k3ntV3HzfzG4Cflhkbqk2n/sR9BsOd0yIpqf0hR88D1tvFzaXdL/WXwRUFGpeqbuSZgMtF9AZD7R3Z5fjaLUbKS4mmJkRHZ94psQ8EtJex8FpjyXTv/iIrq+UZe+/paKQUaUWhmnAwWa2BDg4nsbMGs3s+pZOZjYM2BH471av/52ZLQAWAAMADYqvdYM/1mrE0h3atZRFD06Bn+6QTP/Lf6ooZIi5e+gMndbY2OhNTU2hY0hH7voeNN2YTJ98n856zYLWhX7yWh1krhFmNtfdGzvqpzOfpft88TI4e3kyfdNYOL9/4f5S3V5fklsUhh+gkUcZpcIg3WvL/rm7GHxj9Mdlw/pwmaTzrj0Arkx90fzOfBj/h3B5pFupMEhlTFkHR92QTF/YAE/8KlweKU7L9Y5WzU/mTVmny1tknAqDVM4eR8OkN5Pp+8+N/uiseyVcJinsgobc6x0d+BMdYK4THZ7HIFJWPXpEf1yu+hQ0L47mtZwt+5M3oKf+SQb3h+/A3F/nzjvvVd1DoY7of6GEcfqcaDdF+hvpBdtGz/pWGsbiu+DWr+fO+/ptMOLgMHkkGBUGCccsKgLv/T33wmtT+sJO+8LJ94TLVk/WvgSX75E771PfhrE/DZNHglNhkPA2/3BUIFbNj0a/ALz4p6hAHPpT2OfbYfNl1cYPci+PDdHlsc95IUgcqR46wU2qz5PT4d6zcud98xEY8vEwebIo39noOlEt84o9wU2FQarXjC/B8sdy5w3+WO71mKR4K+bC9Qe2nX/uK9DnQ5XPIxVXbGHQriSpXi0nUKW/3a6an0xPeBwG7dH2dZLrosHwQZ57Y532PzB4z8rnkaqnwiDVr2WU0p+vg3tSV2Zvuf7/VgPhrCWVz1XNlj8WbXHlM+nNaNiwSAHalSS1Z+OGZGhra/V8ob7Ww3/TvvY7GPnFyuaRqqNdSZJdPXslWxELboP/PCVZdtPYpD3+Lhi+f2WzVdqG9fDLj8FbK/Mv1wFl6QJtMUg2bNoEU/u132fCn2DQ7pXJ0102bYp2Eb34eOE+J9wOu+Q5yCx1ryJbDGb2VWAKMBIY4+55/1qb2Vjgl0BP4Hp3b7mhz3BgJtH9nv8CnODuuuymdF7LpTYAXlsIV3+6bZ9r9s2d/u4zsM2ObftVm9lnwl9mtN9nzGlw+KWVySOZV9IWg5mNBDYB1wI/zFcYzKwn8DzRHd5WAE8Bx7n7IjObBfyXu880s2uA+e5+dUefqy0G6ZQXHodff6Hjfv2Gw5evgaGf6v5M+by7Nroz2tybOu474hA4fpZ2E0mnVGSLwd0Xxx/WXrcxwFJ3Xxb3nQmMM7PFwIHA8XG/GURbHx0WBpFOGbZf7vWXFt4Bv89zL+o3l8ONhxZ+n4GjYNxVXT/R7oN34b8vgccv6/xr++8Cpz8JPTfr2meLdEIlDj4PAV5OTa8A9ga2Bda6+4bU/CEVyCP1brcjYbdUoZg/E24/rePXrV4E132u+3KljTgUjr4B+mxdmc8TSemwMJjZg8CgPIvOc/c7i/iMfJsT3s78QjlOBU4FGDp0aBEfK1Kkjx0bPfLZsB6e+Hd4aGp5P3PHveFLV8DAXcv7viJl0GFhcPeDSvyMFUD6CN8OwErgdWAbM+sVbzW0zC+UYzowHaJjDCVmEilOr96w/w+ih0idqMTpj08BI8xsuJn1Bo4FZnt01PsR4Oi433igmC0QERHpRiUVBjP7spmtAPYB7jaz++P525vZPQDx1sAZwP3AYmCWuy+M3+Ic4PtmtpTomMMNrT9DREQqSye4iYjUiWKHq+pKWiIikkOFQUREcqgwiIhIDhUGERHJocIgIiI5anJUkpk1Ay+W6e0GEJ1sV8uysA6QjfXIwjpANtYjC+sA5V2Pndy9oaNONVkYysnMmooZvlXNsrAOkI31yMI6QDbWIwvrAGHWQ7uSREQkhwqDiIjkUGGIL8xX47KwDpCN9cjCOkA21iML6wAB1qPujzGIiEgubTGIiEiOuisMZvZVM1toZpvMrOCRfjN7wcwWmNk8M6uqK/ZNzkt4AAADvElEQVR1Yh3GmtlzZrbUzCZWMmMxzKy/mT1gZkvi534F+m2Mfw/zzGx2pXPm09HP1sz6mNmt8fInzWxY5VO2r4h1OMnMmlM/+2+EyNkeM7vRzFab2TMFlpuZXRGv49Nm1sX7snavItbjs2a2LvW7mNStgdy9rh7ASOCjwKNAYzv9XgAGhM7b1XUAegJ/A3YGegPzgVGhs7fKeCkwMW5PBC4p0O/t0Fk7+7MFvg1cE7ePBW4NnbsL63AScGXorB2sxwHAx4FnCiw/HLiX6I6RnwKeDJ25i+vxWeCuSuWpuy0Gd1/s7s+FzlGKItdhDLDU3Ze5+3pgJjCu+9N1yjhgRtyeARwZMEtnFPOzTa/bbcDnzSzf7WxDqYV/Hx1y98eANe10GQf8xiNziO4aObgy6YpXxHpUVN0Vhk5w4I9mNje+33StGQK8nJpeEc+rJtu5+yqA+HlggX6bm1mTmc0xs2ooHsX8bP/Zx6ObVa0juhlVtSj238dR8S6Y28xsxzzLq10t/D8o1j5mNt/M7jWz3brzgzq853MtMrMHgUF5Fp3n7sXePnRfd19pZgOBB8zs2biqV0QZ1iHft9OKD0Frbz068TZD49/FzsDDZrbA3f9WnoRdUszPtip+/u0oJt8fgFvc/X0zm0C0BXRgtycrr2r/PRTrL0SXs3jbzA4H7gBGdNeHZbIwuPtBZXiPlfHzajO7nWjTu2KFoQzrsAJIf8PbAVhZ4nt2WnvrYWavmdlgd18Vb96vLvAeLb+LZWb2KDCaaP94KMX8bFv6rDCzXkBfqmhXAUWsg7u/kZq8DrikArnKrSr+H5TK3f+eat9jZr8yswHu3i3XgtKupDzMbCsz27qlDRwC5B0tUMWeAkaY2XAz6010ALQqRvSkzAbGx+3xQJstITPrZ2Z94vYAYF9gUcUS5lfMzza9bkcDD3t8FLFKdLgOrfbFH0F0z/ZaMxs4MR6d9ClgXcvuy1piZoNajlGZ2Riiv91vtP+qEoQ+Gl/pB/Blom8R7wOvAffH87cH7onbOxON0pgPLCTafRM8e2fWIZ4+HHie6Nt1Va1DnG9b4CFgSfzcP57fCFwftz8NLIh/FwuAU0LnLvSzBaYCR8TtzYHfA0uBPwM7h87chXX4afzvfz7wCLBr6Mx51uEWYBXwQfx/4hRgAjAhXm7AVfE6LqCdkYhVvh5npH4Xc4BPd2cenfksIiI5tCtJRERyqDCIiEgOFQYREcmhwiAiIjlUGEREJIcKg4iI5FBhEBGRHCoMIiKS4/8ATMvr2fwfp8QAAAAASUVORK5CYII=\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    },
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAa0AAAEKCAYAAAChTwphAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADl0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uIDIuMi4zLCBodHRwOi8vbWF0cGxvdGxpYi5vcmcvIxREBQAAIABJREFUeJzt3Xt0VOW9//H31wQEAblJrSVSULEIgcRkoKWoICKXWsGi9ng5R+Bw12rVVQvU9oeWXqyytNUqyPFSsbZA8QJVqIDCKZyiEBTlohgETo1SQECQmzLw/f0xm5wQZiZDyGSyw+e1Vhazn3n293n2uOTD3rOzH3N3REREwuCUTE9AREQkVQotEREJDYWWiIiEhkJLRERCQ6ElIiKhodASEZHQUGiJiEhoKLRERCQ0FFoiIhIa2ZmeQG1zxhlneOvWrTM9DRGRUFmxYsWn7t6ion4KrSrWunVrioqKMj0NEZFQMbP/TaWfLg+KiEhoKLRERCQ0FFoiIhIaCi0REQkNhZaIiISGQktEREJDoSUiIqGh39OqIZ75xya27/0y09MQEam0W3ueR52s9J4LKbRqiD++8b8Ub92T6WmIiFTazT3OpU5WesdQaNUQ8+/snukpiIjUePpOS0REQkOhJSIioaHQEhGR0FBoiYhIaCi0REQkNBRaIiISGgotEREJDYWWiIiEhkJLRERCQ6ElIiKhodASEZHQUGiJiEhoKLRERCQ0FFoiIhIaCi0REQkNhZaIiISGQktEREIjo6FlZn3NbJ2ZrTezsXHeP9XMpgfvv2lmrcu8Ny5oX2dmfSqqaWZtghrFQc26ycYws+ZmttDM9pjZ79P3KYiISKoyFlpmlgU8CvQD2gPXm1n7ct2GAjvd/TzgIeA3wb7tgeuADkBf4DEzy6qg5m+Ah9y9LbAzqJ1wDOAA8DPgR1V64CIiUmmZPNPqAqx39w3u/iUwDRhQrs8A4Jng9UzgMjOzoH2au3/h7huB9UG9uDWDfXoGNQhqXpVsDHff6+5LiIWXiIjUAJkMrZbAR2W2S4K2uH3cPQrsApon2TdRe3Pgs6BG+bESjZEyMxthZkVmVrRt27bj2VVERI5DJkPL4rR5in2qqj3VeSTl7lPcPeLukRYtWhzPriIichwyGVolwNlltnOATxL1MbNsoDGwI8m+ido/BZoENcqPlWgMERGpYTIZWsuBtsFdfXWJ3Vgxu1yf2cCg4PU1wOvu7kH7dcGdf22AtsCyRDWDfRYGNQhqzqpgDBERqWGyK+6SHu4eNbMfAK8CWcBT7r7GzH4OFLn7bOBJ4FkzW0/s7Oe6YN81ZjYDWAtEgVvc/RBAvJrBkGOAaWb2C+DtoDaJxghqbQJOB+qa2VVAb3dfm55PREREKmI6qahakUjEi4qKMj0NEZFQMbMV7h6pqJ+eiCEiIqGh0BIRkdBQaImISGgotEREJDQUWiIiEhoKLRERCQ2FloiIhIZCS0REQkOhJSIioaHQEhGR0FBoiYhIaCi0REQkNBRaIiISGgotEREJDYWWiIiEhkJLRERCQ6ElIiKhodASEZHQUGiJiEhoKLRERCQ0FFoiIhIaCi0REQkNhZaIiISGQktEREJDoSUiIqGh0BIRkdBQaImISGgotEREJDQUWiIiEhoKLRERCQ2FloiIhIZCS0REQkOhJSIioaHQEhGR0MhoaJlZXzNbZ2brzWxsnPdPNbPpwftvmlnrMu+NC9rXmVmfimqaWZugRnFQs25lxxARkczIWGiZWRbwKNAPaA9cb2bty3UbCux09/OAh4DfBPu2B64DOgB9gcfMLKuCmr8BHnL3tsDOoPZxj1G1n4KIiByPTJ5pdQHWu/sGd/8SmAYMKNdnAPBM8HomcJmZWdA+zd2/cPeNwPqgXtyawT49gxoENa+q5BgiIpIhmQytlsBHZbZLgra4fdw9CuwCmifZN1F7c+CzoEb5sY53DBERyZBMhpbFafMU+1RVe2XGOIaZjTCzIjMr2rZtW7wuIiJSBTIZWiXA2WW2c4BPEvUxs2ygMbAjyb6J2j8FmgQ1yo91vGMcw92nuHvE3SMtWrRIetAiIlJ5mQyt5UDb4K6+usRuephdrs9sYFDw+hrgdXf3oP264M6/NkBbYFmimsE+C4MaBDVnVXIMERHJkOyKu6SHu0fN7AfAq0AW8JS7rzGznwNF7j4beBJ41szWEzv7uS7Yd42ZzQDWAlHgFnc/BBCvZjDkGGCamf0CeDuoTWXGEBGRzLDYSYVUlUgk4kVFRZmehohIqJjZCnePVNRPT8QQEZHQUGiJiEhoKLRERCQ0FFoiIhIaGbt7UEQkmYMHD1JSUsKBAwcyPRWpQvXq1SMnJ4c6depUan+FlojUSCUlJTRq1IjWrVsTexyohJ27s337dkpKSmjTpk2laujyoIjUSAcOHKB58+YKrFrEzGjevPkJnT0rtESkxlJg1T4n+t9UoSUikkDDhg1LX8+ZM4e2bdvyz3/+k8mTJzN16tRK1dy0aRN/+tOfSreLioq47bbbTniuJwt9pyUiUoHXXnuNW2+9lXnz5tGqVStGjRpV6VpHQuuGG24AIBKJEIlU+CCItDp06BBZWeFY41ZnWiIiSSxevJjhw4fzyiuvcO655wJwzz33MHHiRAB69OjBmDFj6NKlC+effz6LFy8GYkFw11130blzZzp16sTjjz8OwNixY1m8eDH5+fk89NBDLFq0iO9+97sA7NmzhyFDhtCxY0c6derE888/f8x8VqxYQffu3SksLKRPnz5s3ry5UvNYtGgRl156KTfccAMdO3YEYMKECbRr147LL7+c66+/nokTJ/Lhhx9SUFBQOn5xcTGFhYVV/jmnSmdaIiIJfPHFFwwYMIBFixbRrl27hP2i0SjLli1jzpw53HvvvSxYsIAnn3ySxo0bs3z5cr744gu6detG7969ue+++5g4cSIvv/wyEAuPIyZMmEDjxo1ZtWoVADt37jxqnIMHD3Lrrbcya9YsWrRowfTp07n77rt56qmnjnseAMuWLWP16tW0adOGoqIinn/+ed5++22i0SgFBQUUFhZy7rnn0rhxY1auXEl+fj5PP/00gwcPrsJP+fgotESkxrv3r2tY+8nuKq3Z/munM/7KDkn71KlTh29/+9s8+eST/O53v0vYb+DAgQAUFhayadMmAObNm8e7777LzJkzAdi1axfFxcXUrVs3YZ0FCxYwbdq00u2mTZse9f66detYvXo1l19+ORA7izrrrLMqPY8uXbqU3nq+ZMkSBgwYQP369QG48sorS+sOGzaMp59+mgcffJDp06ezbFnmVmlKKbTM7HngKWCuux9O75RERGqGU045hRkzZtCrVy9+9atf8ZOf/CRuv1NPPRWArKwsotEoEPudpEceeYQ+ffoc1bfsmVV57p707jp3p0OHDixdurRK5tGgQYOjaidy9dVXc++999KzZ08KCwtp3rx5wr7pluqZ1iRgCPCwmf0F+IO7v5++aYmI/J+KzojS6bTTTuPll1/m4osv5swzz2To0KEp7denTx8mTZpEz549qVOnDh988AEtW7akUaNGfP7553H36d27N7///e/57W9/C8QuD5Y92/rGN77Btm3bWLp0KV27duXgwYN88MEHdOiQ+PNJNI/yLrroIkaOHMm4ceOIRqO88sorDB8+HIg9xaJPnz6MHj2aJ5988ph9q1NKN2K4+wJ3vxEoADYB883sH2Y2xMwq9ywOEZGQaNasGX/729/4xS9+waxZsyregdgltfbt21NQUEBubi4jR44kGo3SqVMnsrOzycvL46GHHjpqn5/+9Kfs3LmT3Nxc8vLyWLhw4VHv161bl5kzZzJmzBjy8vLIz8/nH//4R6XmUV7nzp3p378/eXl5DBw4kEgkQuPGjUvfv/HGGzGz0u/DMiXlRSDNrDnw78B/AJ8AzwEXAR3dvUe6Jhg2WgRSpGq89957XHDBBZmexkllz549NGzYkH379nHJJZcwZcqU0jsHJ06cyK5du5gwYcIJjxPvv22qi0Cm+p3WC0A74FngSnffHLw13cz0N7SISC0wYsQI1q5dy4EDBxg0aFBpYH3ve9/jww8/5PXXX8/wDFP/Tuv37h53tqkko4iI1Hxln9RR1osvvljNM0ks1dBqYmYDy7XtAla5+9YqnpOIiEhcqYbWUKArcORbwR7AG8D5ZvZzd382DXMTERE5SqqhdRi4wN23AJjZmcRug/8m8Hdi33WJiIikVarPHmx9JLACW4Hz3X0HcLDqpyUiInKsVENrsZm9bGaDzGwQMAv4u5k1AD5L3/RERDInKyuL/Px8cnNzufbaa9m3b1/Cvp999hmPPfbYCY33hz/8gU8++aR0e9iwYaxduzbpPj169OBk+jWbVEPrFuBpIB+4EJgK3OLue9390nRNTkQkk+rXr8/KlStZvXo1devWZfLkyQn7piO0nnjiCdq3b39CNavSoUOHMj2FikPLzLKA+e7+vLvf4e63u/tMT/W3kkVEaoGLL76Y9evXA/Dggw+Sm5tLbm5u6SOXxo4dy4cffkh+fj533XUXAA888EDpkiDjx48HYutpXXDBBQwfPpwOHTrQu3dv9u/fz8yZMykqKuLGG28kPz+f/fv3H3UWNXr0aCKRCB06dCitlUytXcLE3Sv8AWYDjVPpe7L/FBYWuoicuLVr12Z6Ct6gQQN3dz948KD379/fH3vsMS8qKvLc3Fzfs2ePf/75596+fXt/6623fOPGjd6hQ4fSfV999VUfPny4Hz582A8dOuRXXHGF//d//7dv3LjRs7Ky/O2333Z392uvvdafffZZd3fv3r27L1++vLRG2e3t27e7u3s0GvXu3bv7O++8E3cfd/cvv/zSu3bt6lu3bnV392nTpvmQIUNK+995553u7v7KK6/4ZZdd5u7ujz/+uE+YMMHd3Q8cOOCFhYW+YcMGX7hwoZ922mm+YcMGd3dfvny55+Xl+b59+3z37t1+3nnn+QMPPODu7j169Cg9rnHjxvnDDz8c93ON998WKPIU/o5N9e7BA8AqM5sP7C0TeFojWkTSb+5Y+Neqqq351Y7Q776kXfbv309+fj4QO9MaOnQokyZN4nvf+17pE9IHDhzI4sWL6d+//1H7zps3j3nz5nHhhRcCsUckFRcX06pVK9q0aVNat+wyIsnMmDGDKVOmEI1G2bx5M2vXrqVTp05x+9bmJUxSDa1Xgh8RkZPGke+0yvIUvxlxd8aNG8fIkSOPat+0aVPpEiIQu9lj//79SWtt3LiRiRMnsnz5cpo2bcrgwYM5cOBA0rFr6xImKYWWuz9jZvWBVu6+rspnISKSTAVnRNXpkksuYfDgwYwdOxZ358UXX+TZZ589ZsmRPn368LOf/Ywbb7yRhg0b8vHHH1OnTvJFMRItW7J7924aNGhA48aN2bJlC3PnzqVHjx4J69TmJUxSfWDulcBEoC7QxszygZ+7e//ke4qI1C4FBQUMHjyYLl26ALFLYkcuAXbr1o3c3Fz69evHAw88wHvvvUfXrl0BaNiwIX/84x/JyspKWHvw4MGMGjWK+vXrH3WWlJeXx4UXXkiHDh0455xz6NatW9I5HlnC5LbbbmPXrl1Eo1Fuv/32pKE1bNgwNm3aREFBAe5OixYteOmll47pV3YJk69//etxlzB54YUX0raESUpLk5jZCqAnsMjdLwzaVrl7x7TMKsS0NIlI1dDSJDXXiS5hkvalSYCou+8qtwy0bnkXETkJZXIJk1RDa7WZ3QBkmVlb4DYg+XKZIiJSK2VyCZNUn4hxK9AB+AL4M7AbuD1dkxIREYknpdBy933ufre7d3b3SPA68f2WFTCzZmY238yKgz+bJug3KOhTHDzz8Eh7oZmtMrP1ZvawBdctE9W1mIeD/u+aWUEKY/zSzD4ysz2VPU4REalaKYWWmZ1vZlPMbJ6ZvX7k5wTGHQu85u5tgdeC7fJjNgPGE1v+pAswvky4TQJGAG2Dn74V1O1Xpu+IYP+Kxvhr0CYiIjVEqt9p/QWYDDwBVMUTEwcQW0gS4BlgETCmXJ8+xJ55uAMgeBpHXzNbBJzu7kuD9qnAVcDcJHUHAFODR4W8YWZNzOysoO8xYwB/dvc3grYqOFwREakKx3P34KQqHPdMd98M4O6bzewrcfq0BD4qs10StLUMXpdvT1Y3Wa147SJyktu+fTuXXXYZAP/617/IysqiRYsWACxbtoy6dese1X/Hjh3MmDGDUaNGJa0bjUY544wz+OwzrepUGamG1l/N7GbgRWI3YwBw5AwlHjNbAHw1zlt3pzhmvFMcT9JeXbWOLW42gthlR1q1anW8u4tIDdS8efPSRzjdc889NGzYkB/96EcJ++/YsYPJkydXGFrVLRqNkp2d6l/1NV+qdw8OAu4idpv7iuAn6W/Qunsvd8+N8zML2BJcniP4c2ucEiXA2WW2c4BPgvacOO0kqZusVrz24+LuU4IbVCJH/iUmIrXX/fffX7o0ySOPPALEliZZt24d+fn5jB07lt27d9OzZ08KCgro1KkTL7/8coV1n3nmGbp06UJ+fj4333wzhw8fJhqN0qRJE8aOHUteXh5du3Zl69bYX21btmxh4MCBRCIRunTpwhtvvAHAT3/6U0aOHMnll1/OkCFD2Lt3L1dffTV5eXlcf/31RCIRVq5cyeOPP166jArApEmT+PGPf5yGT6wKpfIo+Kr+AR4AxgavxwL3x+nTDNgINA1+NgLNgveWA98idqY0F/hOsrrAFUE/C/ZbVtEYZeax53iOTUuTiFSNmrA0yRHjx48vXX7jzTff9E6dOvnevXt99+7d3q5dO3/nnXe8uLjY8/LySvf58ssvfffu3e7uvmXLFj/vvPPcPbbMSePGjY8ZY9WqVT5gwAA/ePCgu7sPHz7cn3vuOT948KADPmfOHHd3v+OOO/zXv/61u7t///vf96VLl7q7H7U0yt133+2dO3f2/fv3u7v7r3/9a7/55pvd3X3lypV+yimn+Ntvv+27d+/2c845p3TMzp07+5o1a6rwk4svbUuTmNmP3f3+4PW17v6XMu/9yt1/Urmo5D5ghpkNBf4JXBvUjACj3H2Yu+8wswlBQEHsWYdHLkeOBv4A1A/CaG6yusAc4DvAemAfMARilzcTjWFm9wM3AKeZWQnwhLvfU8njFZET8Jtlv+H9He9Xac12zdoxpkv5+78qtnjxYq6++mpOO+00AK666iqWLFlyzLP23J0xY8awZMkSTjnlFD766CM+/fRTmjRpErfuggULWL58OZFI7ElG+/fv5+yzYxeC6tevT79+/YDYciJHFm5csGAB69b93zPMd+7cWfrE+AEDBlCvXj0gtpzImDGxY83Lyyt9BmGjRo245JJLmDt3Lueccw5ZWVk1aqXkeCq60HkdcH/wehyxuwiP6AtUKrTcfTtwWZz2ImBYme2ngKcS9Ms9jroO3JJgLonG+DFQw8+TRaS6eYpLk0ydOpVdu3bx1ltvkZ2dTU5OToXLifznf/7nMc/si0ajR930UX45kXg3hQApLycybNgwHnzwQVq3bs2QIUNSOrZMqii0LMHreNsiImlRmTOidLnkkksYOXIkd911F4cOHWLWrFlMnz79mGVFdu3axVe+8hWys7OZP38+H3/8cdK6vXr14pprruGHP/whZ5xxBtu3b2fv3r187WtfS7rPo48+yh133AHAypUrSxeXLOuiiy5ixowZXHzxxaxatYq1a9eWvtetWzduueUWli9fzqpVVbzQZhpUFFqe4HW8bRGRWq9Lly5cf/31dO7cGYDRo0fTsWNswYtIJELHjh254ooruPPOO7nyyiuJRCIUFBTQtm3bpHU7duzI+PHj6dWrF4cPH6ZOnTpMnjw5aWg9+uijjB49mqeffppoNMqll17Ko48+eky/W2+9lZtuuolOnTpRUFBAbm7uUcuJXHPNNbz//vtHtdVUSZcmMbNDwF5iZ1X1iX0fRLBdz92Tr2h2EtLSJCJVQ0uTVJ1oNEo0GqVevXoUFxfTu3dviouLS2+F79u3L+PGjaN79+7VMp+0LU3i7olXKxMRkVDYs2cPl112GdFoFHfn8ccfJzs7m+3bt9O1a1cKCwurLbBOVO35jTMREYmrSZMmrFix4pj25s2b88EHH2RgRpWX6i8Xi4iIZJxCS0RqrFRvL5fwONH/pgotEamR6tWrx/bt2xVctYi7s3379tJfeq4MfaclIjVSTk4OJSUlbNu2LdNTkSpUr149cnJyKu6YgEJLRGqkOnXq0KZNm0xPQ2oYXR4UEZHQUGiJiEhoKLRERCQ0FFoiIhIaCi0REQkNhZaIiISGbnmvIdKxMquISHWp7ErQx0tnWiIiEho606ohatLKrCIiNZXOtEREJDQUWiIiEhoKLRERCQ2FloiIhIZCS0REQkOhJSIioaHQEhGR0FBoiYhIaCi0REQkNBRaIiISGgotEREJDYWWiIiEhkJLRERCQ6ElIiKhodASEZHQUGiJiEhoZCS0zKyZmc03s+Lgz6YJ+g0K+hSb2aAy7YVmtsrM1pvZw2ZmyepazMNB/3fNrCDZGGZ2mpm9Ymbvm9kaM7svvZ+IiIikIlNnWmOB19y9LfBasH0UM2sGjAe+CXQBxpcJt0nACKBt8NO3grr9yvQdEexf0RgT3b0dcCHQzcz6Vc2hi4hIZWUqtAYAzwSvnwGuitOnDzDf3Xe4+05gPtDXzM4CTnf3pe7uwNQy+yeqOwCY6jFvAE2COnHHcPd97r4QwN2/BN4Ccqrs6EVEpFIyFVpnuvtmgODPr8Tp0xL4qMx2SdDWMnhdvj1Z3WS14rWXMrMmwJXEztxERCSDstNV2MwWAF+N89bdqZaI0+ZJ2qu8lpllA38GHnb3DQmLm40gdtmRVq1aVTAVERGprLSFlrv3SvSemW0xs7PcfXNwmW5rnG4lQI8y2znAoqA9p1z7J8HrRHVLgLPj7JNojCOmAMXu/ttExwLg7lOCvkQikYoCVEREKilTlwdnA0fuBhwEzIrT51Wgt5k1DW6O6A28Glz2+9zMvhXcNXhTmf0T1Z0N3BTcRfgtYFdQJ+4YAGb2C6AxcHuVHbWIiJyQTIXWfcDlZlYMXB5sY2YRM3sCwN13ABOA5cHPz4M2gNHAE8B64ENgbrK6wBxgQ9D/v4Cbk41hZjnELmO2B94ys5VmNiwdH4SIiKTOYjfgSVWJRCJeVFSU6WmIiISKma1w90hF/fREDBERCQ2FloiIhIZCS0REQkOhJSIioaHQEhGR0FBoiYhIaCi0REQkNBRaIiISGgotEREJDYWWiIiEhkJLRERCQ6ElIiKhodASEZHQUGiJiEhoKLRERCQ0FFoiIhIaCi0REQkNhZaIiISGQktEREJDoSUiIqGh0BIRkdBQaImISGgotEREJDQUWiIiEhoKLRERCQ2FloiIhIZCS0REQkOhJSIioaHQEhGR0FBoiYhIaCi0REQkNBRaIiISGgotEREJDYWWiIiEhkJLRERCIyOhZWbNzGy+mRUHfzZN0G9Q0KfYzAaVaS80s1Vmtt7MHjYzS1bXYh4O+r9rZgUpjPE3M3vHzNaY2WQzy0rfJyIiIqnI1JnWWOA1d28LvBZsH8XMmgHjgW8CXYDxZcJtEjACaBv89K2gbr8yfUcE+1c0xvfdPQ/IBVoA11bJkYuISKVlKrQGAM8Er58BrorTpw8w3913uPtOYD7Q18zOAk5396Xu7sDUMvsnqjsAmOoxbwBNgjpxxwBw993BvtlAXcCr4sBFRKTysjM07pnuvhnA3Teb2Vfi9GkJfFRmuyRoaxm8Lt+erG6yWvHaATCzV4mdgc0FZh7PAR63P/0b7NiQ1iFERNJq1BLIPjWtQ6QttMxsAfDVOG/dnWqJOG2epL3Ka7l7HzOrBzwH9CR2JnZscbMRxC470qpVqwqmkkDz86BO/crtKyJSI8T7K7VqpS203L1XovfMbIuZnRWcDZ0FbI3TrQToUWY7B1gUtOeUa/8keJ2obglwdpx9Eo1R9jgOmNlsYpcY44aWu08BpgBEIpHKXUbs88tK7SYicjLJ1Hdas4Ejd+oNAmbF6fMq0NvMmgY3R/QGXg0u/31uZt8K7hq8qcz+ierOBm4K7iL8FrArqBN3DDNrGIQeZpYNfAd4v8qOXkREKiVT32ndB8wws6HAPwnuzDOzCDDK3Ye5+w4zmwAsD/b5ubvvCF6PBv4A1Cf2fdPcZHWBOcSCZz2wDxgCkGgMMzsTmG1mpwJZwOvA5Cr+DERE5DhZ7AY8qSqRSMSLiooyPQ0RkVAxsxXuHqmon56IISIioaHQEhGR0FBoiYhIaCi0REQkNBRaIiISGrp7sIqZ2Tbgfyu5+xnAp1U4nTDQMZ8cdMwnj8oe99fdvUVFnRRaNYiZFaVyy2dtomM+OeiYTx7pPm5dHhQRkdBQaImISGgotGqWKZmeQAbomE8OOuaTR1qPW99piYhIaOhMS0REQkOhVQOYWV8zW2dm681sbKbnUx3M7Ckz22pmqzM9l+piZmeb2UIze8/M1pjZDzM9p3Qzs3pmtszM3gmO+d5Mz6m6mFmWmb1tZi9nei7Vwcw2mdkqM1tpZml7arguD2aYmWUBHwCXE1uUcjlwvbuvzejE0szMLgH2AFPdPTfT86kOwRptZ7n7W2bWCFgBXFWb/1sHa941cPc9ZlYHWAL80N3fyPDU0s7M7gQiwOnu/t1MzyfdzGwTEHH3tP5ums60Mq8LsN7dN7j7l8A0Yqsk12ru/ndgR4UdaxF33+zubwWvPwfeA1pmdlbp5TF7gs06wU+t/5eymeUAVwBPZHoutY1CK/NaAh+V2S6hlv9FJmBmrYELgTczO5P0Cy6TrQS2AvPdvdYfM/Bb4MfA4UxPpBo5MM/MVpjZiHQNotDKPIvTVuv/JXoyM7OGwPPA7e6+O9PzSTd3P+Tu+UAO0MXMavXlYDP7LrDV3Vdkei7VrJu7FwD9gFuCrwCqnEIr80qAs8ts5wCfZGgukmbB9zrPA8+5+wuZnk91cvfPgEVA3wxPJd26Af2D73imAT3N7I+ZnVL6ufsnwZ9bgReJffVR5RRambccaGtmbcysLnAdMDvDc5I0CG5KeBJHLc1vAAADl0lEQVR4z90fzPR8qoOZtTCzJsHr+kAv4P3Mziq93H2cu+e4e2ti/z+/7u7/nuFppZWZNQhuLsLMGgC9gbTcGazQyjB3jwI/AF4l9sX8DHdfk9lZpZ+Z/RlYCnzDzErMbGim51QNugH/Qexf3iuDn+9kelJpdhaw0MzeJfYPtPnuflLcAn6SORNYYmbvAMuAV9z9b+kYSLe8i4hIaOhMS0REQkOhJSIioaHQEhGR0FBoiYhIaCi0REQkNBRaItXMzJqY2c1ltr9mZjPTNNZVZvb/0lG7gnHrmtnfzSy7useW2k23vItUs+C5gy9Xx9PtzewfQP90PXnbzLKD3zWM9954Yg+Dfi4dY8vJSWdaItXvPuDc4JeLHzCz1kfWFTOzwWb2kpn91cw2mtkPzOzOYF2mN8ysWdDvXDP7W/Bw0sVm1q78IGZ2PvCFu39qZo2CenWC904P1j+qk6iWmV1pZm8GYy8wszOD9nvMbIqZzQOmmlmHYM2slWb2rpm1DabwEnBj2j9NOakotESq31jgQ3fPd/e74ryfC9xA7NltvwT2ufuFxJ4gclPQZwpwq7sXAj8CHotTpxtQdimURcSWy4DY44Wed/eDSWotAb4VjD2N2FPLjygEBrj7DcAo4HfBQ3EjxJ6nCbHH+HRO6RMRSZGuN4vUPAuDkPnczHYBfw3aVwGdgqfEfxv4S+xxhgCcGqfOWcC2MttPEAuel4AhwPAKauUA04PFK+sCG8vUmu3u+4PXS4G7gzWkXnD3Yog93d3MvjSzRsHxiJwwnWmJ1DxflHl9uMz2YWL/0DwF+Cw4Uzvyc0GcOvuBekc23P1/gNZm1h3IcvfVFdR6BPi9u3cERpatBewtU/dPQP9gvFfNrGeZfqcCB473AxBJRKElUv0+BxpVdudgDa6NZnYtxJ4eb2Z5cbq+B5xXrm0q8Gfg6RRqNQY+Dl4PSjQfMzsH2ODuDxNboaBT0N4c2BZcghSpEgotkWrm7tuB/zGz1Wb2QCXL3AgMDZ6qvQYYEKfP34ELrcx1P+A5oCmx4Kqo1j3ELhsuBpLdffhvwOpgdeJ2xIIR4FJgzvEclEhFdMu7SC1mZr8D/uruC4Lta4jdQPEf1TD2C8A4d1+X7rHk5KEbMURqt18B3wQws0eILYWe9jW8ggVNX1JgSVXTmZaIiISGvtMSEZHQUGiJiEhoKLRERCQ0FFoiIhIaCi0REQkNhZaIiITG/wdjL0AfpfvRSAAAAABJRU5ErkJggg==\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    },
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAaQAAAEKCAYAAABdWiGrAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADl0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uIDIuMi4zLCBodHRwOi8vbWF0cGxvdGxpYi5vcmcvIxREBQAAIABJREFUeJzt3Xd8VvX5//HXRdh7IyuEDWEoEMBVqbiwVUHFinsWbbW2X3/KcFRFqqKt1tZVrPpVa4sKKDgRBfeA4EhYkRBWABmGPULG9fvjPnwb453kBnPnvpO8n49HHpxz7s+5znUHyJVzzue+jrk7IiIisVYj1gmIiIiACpKIiMQJFSQREYkLKkgiIhIXVJBERCQuqCCJiEhcUEESEZG4oIIkIiJxQQVJRETiQs1YJ1CZtGzZ0pOSkmKdhohIpbJo0aKt7t6qrHEqSIcgKSmJ1NTUWKchIlKpmNmaSMbpkp2IiMQFFSQREYkLKkgiIhIXVJBERCQuqCCJiEhcUEESEZG4oIIkIiJxQQVJRERKNXfpJl5cuDbqx9EHY0VEJKytu3O5c/YSXk/byMDEppw3qCM1aljUjqeCJCIiP+DuvPr1eu56bSl7cwu46dQeXDOsa1SLEaggiYhIERu27+PWV9KZn7GFgYlNuX90f7q1blQhx1ZBEhERCgudFxas5b43l1HocMeZyVx6TBIJUT4rKkoFSUSkmsvaspsJM9JZsDqH47u15N5z+tGxef0Kz0MFSUSkmsovKOSfH6/iobnfUqdmDe4f3Z/zBnXArOLOiopSQRIRqYaWbtjJuBnfsHj9Tk7r04a7R/aldeO6Mc1JBUlEpBrJzS/gkXmZPP7+SprWr8VjFw3k9L5HxOysqCgVJBGRamLRmhzGTU9j5ZY9nDuwA7ef0Zum9WvHOq3/o4IkIlLF7cnN54E5GTz72WraNanHs1cOYViPMp8oXuFUkEREqrCPVmxh4sx0srft47JjOnHziF40rBOfP/rjMysREflJduzNY/IbS3l5UTZdWjXg5WuPYXBS81inVSoVJBGRKubtxd9x+6zF5Ow5wG9/3pUbTupO3VoJsU6rTFHt9m1mI8wsw8wyzWxCmNfrmNmLwetfmFlSkdcmBtszzOy0smKaWecgxoogZu1g+41mttTM0szsPTPrVGSfAjP7OviaHa3vg4hIRdi8az+/fWER1/5rEa0a1mHWdccxbkSvSlGMIIoFycwSgEeB04Fk4AIzSy427Cpgm7t3Ax4CpgT7JgNjgD7ACOAxM0soI+YU4CF37w5sC2IDfAWkuHt/YDpwf5Hj73P3o4Kvs8rx7YuIVBh3Z/qibE558EPeXbaZm0/ryazrj6Nv+yaxTu2QRPMMaQiQ6e5Z7n4AmAaMLDZmJPBssDwdOMlCk+FHAtPcPdfdVwGZQbywMYN9hgcxCGKOAnD3+e6+N9j+OdAhCu9VRCQmsrft5bJnFnLTy9/QvXVD3rzhZ1x3YjdqJVS+x91F8x5Se2BdkfVsYGhJY9w938x2AC2C7Z8X27d9sBwuZgtgu7vnhxlf1FXAW0XW65pZKpAP3OfurxbfwczGAmMBEhMTw75REZGKVljoPP/5Gqa8vRyAu87qwyVHd4r6IyKiKZoFKdx3xSMcU9L2cCW/tPH/PZDZxUAKMKzI5kR332BmXYB5Zpbu7it/EMR9KjAVICUlpXj+IiIVbuWW3Yyfnkbqmm2c0KMV95zdlw7NKr4ZanmLZkHKBjoWWe8AbChhTLaZ1QSaADll7Btu+1agqZnVDM6SfnAsMzsZuBUY5u65B7e7+4bgzywzex8YAPygIImIxIu8gkKmfpjFw++toF6tBP5y3pGcM7B9XLT9KQ/RvMi4EOgezH6rTWiSQvGZbLOBy4Ll0cA8d/dg+5hgFl5noDuwoKSYwT7zgxgEMWcBmNkA4B/AWe6++eCBzayZmdUJllsCxwFLy/U7ICJSThav38HIRz7hgTkZnNy7Ne/eOIxzY9iZOxqidoYU3BO6HpgDJABPu/sSM5sEpLr7bOAp4HkzyyR0ZjQm2HeJmb1EqEDkA9e5ewFAuJjBIccD08xsMqGZdU8F2x8AGgIvB39xa4MZdb2Bf5hZIaHCfJ+7qyCJSFzZn1fAw++tYOqHWTRvUJsnLh7IiL5tY51WVFjo5EIikZKS4qmpqbFOQ0SqiYWrcxg/PY2srXs4b1AHbvtlMk3q14p1WofMzBa5e0pZ49SpQUQkzuzOzef+t5fz3Gdr6NCsHs9fNYSfdY+/ZqjlTQVJRCSOfPDtFm6Zmc6GHfu4/Ngkbj6tJw3itBlqease71JEJM5t23OAu99Yyswv19O1VQOmX3sMgzrFdzPU8qaCJCISQ+7OW4u/44+zFrN9bx6/G96N64d3o07NytF/rjypIImIxMjmnfu5fdZi5izZRL/2TXjuyqEkt2sc67RiRgVJRKSCuTsvL8pm8utLyc0vZMLpvbj6+M7UrIT958qTCpKISAVal7OXiTPT+ThzK0OSmnPfuf3o0qphrNOKCypIIiIVoKDQee6z1dz/dgYJNYy7R/XloiGJlboZanlTQRIRibIVm3YxfkYaX67dzs97tuKes/vRrmm9WKcVd1SQRESiJK+gkCfeX8nf52XSoE4Cfz3/KEYe1a5K9Z8rTypIIiJRkJ69g5unf8Py73ZxRv+23HlWH1o2rBPrtOKaCpKISDnan1fAQ+9+y5MfZtGyYR2mXjKIU/scEeu0KgUVJBGRcvJF1vdMmJnOqq17GDO4IxN/0Zsm9SpfM9RYUUESEfmJdu3PY8rby/nX52tJbF6fF64eynHdWsY6rUpHBUlE5CeYv3wzt7ySzqad+7n6+M7ceGoP6tfWj9bDoe+aiMhhyNlzgEmvLeHVrzfQvXVDHvvNsQxIbBbrtCo1FSQRkUPg7ryetpE7Zy9hx748fn9Sd357Ytdq2Qy1vKkgiYhEaNPO/dz6ymLeXbaJ/h2a8MKvh9LriOrbDLW8qSCJiJTB3Xlx4Tr+9OYyDuQXcusvenPFcUnVvhlqeVNBEhEpxZrv9zBxZjqfrvyeo7s0575z+pPUskGs06qSSi1IZpYA3ODuD1VQPiIicaGg0Hnmk1X8+Z0MatWowT1n92PM4I5qhhpFpRYkdy8ws5GACpKIVBsZ3+1i3Iw0vlm3nZN6tWby2X1p20TNUKMtkkt2n5jZI8CLwJ6DG939y6hlJSISAwfyC3ns/UwenZ9Jo7q1eHjMUZx1pJqhVpRICtKxwZ+TimxzYHj5pyMiEhvfrNvOuOlpZGzaxcij2vHHM5JpoWaoFarMguTuJ1ZEIiIisbDvQAEPzs3gqY9X0bpRXf55aQonJ7eJdVrVUpkFycyaAHcAJwSbPgAmufuOaCYmIhJtn67cyoQZ6azN2cuFQxOZcHovGtdVM9RYieSS3dPAYuBXwfolwDPAOdFKSkQkmnbuz+PeN5fznwVr6dSiPv/59dEc07VFrNOq9iL5VFdXd7/D3bOCr7uALpEEN7MRZpZhZplmNiHM63XM7MXg9S/MLKnIaxOD7RlmdlpZMc2scxBjRRCzdrD9RjNbamZpZvaemXUqss9lwfgVZnZZJO9JRCq3d5du4pQHP+DFhWsZe0IX3v79CSpGcSKSgrTPzI4/uGJmxwH7ytop+AzTo8DpQDJwgZklFxt2FbDN3bsRmlo+Jdg3GRgD9AFGAI+ZWUIZMacAD7l7d2BbEBvgKyDF3fsD04H7g2M0J3QpcigwBLjDzNQZUaSK+n53Ljf85yuufi6VZvVr88pvj+OWX/SmXm31oIsXkVyyuxZ4LriXBKEf9pGcTQwBMt09C8DMpgEjgaVFxowE7gyWpwOPWGh+5UhgmrvnAqvMLDOIR7iYZraM0Ky/C4MxzwZxH3f3+UWO9zlwcbB8GjDX3XOCWHMJFb//RPDeRKSScHdmf7OBO2cvYXduPv9zcg9+8/Ou1K6ptj/xpqxODTWAnu5+pJk1BnD3nRHGbg+sK7KeTehsJOwYd883sx1Ai2D758X2bR8sh4vZAtju7vlhxhd1FfBWKfmF20dEKqmNO/Zx2yuLeW/5Zo7q2JT7R/enR5tGsU5LSlBWp4ZCM7seeOkQCtFB4T5J5hGOKWl7uF9pShv/3wOZXQykAMMOIT/MbCwwFiAxMTHMLiISbwoLnf8sXMu9by6noNC5/YxkLj82iQS1/YlrkVyym2tmN/HjTg05ZeyXDXQsst4B2FDCmGwzqwk0AXLK2Dfc9q1AUzOrGZwl/eBYZnYycCswLLgMePDYPy8W6/3ib8LdpwJTAVJSUn5UsEQkvqzauocJM9L4YlUOx3Vrwb1n9yexRf1YpyURiKQgXRn8eV2RbU7ZM+0WAt3NrDOwntAkhQuLjZlN6H7UZ8BoYJ67u5nNBv5tZg8C7YDuwAJCZzU/ihnsMz+IMS2IOQvAzAYA/wBGuPvmIseeA9xTZCLDqcDEsr4ZIhKf8gsKefqTVfzlnW+pXbMGU87tx69SOqrtTyUSyT2ki939k0MNHNwTup7QD/4E4Gl3X2Jmk4BUd58NPAU8H0xayCFUYAjGvURoAkQ+cJ27FwQ5/ShmcMjxwDQzm0xoZt1TwfYHgIbAy8E/zLXufpa755jZ3YQKJ4Q+7FvWWZ+IxKFlG3cyfkYaadk7OCW5DZNH9aVN47qxTksOkbmXfhXKzD5z92MqKJ+4lpKS4qmpqbFOQ0QCufkFPDp/JY/Nz6RJvVrcNbIPv+zXVmdFccbMFrl7SlnjIrlk946ZnQvM9LKql4hIBfly7TbGT09jxebdnDOgPbefkUyzBrVjnZb8BJEUpBuBBkCBme0jdB/H3V0PkheRCrf3QD5/nvMtz3y6iraN6/LMFYM5sWfrWKcl5SCSbt+atC8iceGTzK1MmJnGupx9XHJ0J8aN6EkjNUOtMiLp9m3ARUBnd7/bzDoCbd19QdSzExEBduzL4543lvFi6jo6t2zAi2OPZmgX9Z+raiK5ZPcYUEioNc/dwG5C/eQGRzEvEREA3lnyHbe9upjv9xzg2mFd+cPJ3albS/3nqqJICtJQdx9oZl8BuPu2g520RUSiZcuuXO58bQlvpG2kd9vGPHXZYPp1aFL2jlJpRVKQ8oIu2w5gZq0InTGJiJQ7d+eVr9Yz6fWl7M0t4KZTe3DNsK7USlAz1KoukoL0N+AVoLWZ/YlQN4TbopqViFRL67fv49ZX0nk/YwsDE0PNULu11ryq6iKSWXYvmNki4CRCU75HufuyqGcmItVGYaHzwhdruO+t5Thw55nJXHKMmqFWN5GcIeHuy4HlUc5FRKqhrC27mTAjnQWrc/hZ95bcc3Y/OjZXM9TqKKKCJCJS3vILCnnyo1U89O631K1ZgwdG92f0oA5q+1ONqSCJSIVbsmEH42eksXj9Tk7r04a7R/altZqhVnuRfDD2euAFd99WAfmISBW2P6+Av89bwRMfZNGsfm0ev2ggp/drG+u0JE5EcoZ0BLDQzL4EngbmqMmqiByqRWtyGDc9jZVb9nDuwA7cfkZvmtbXRxrlvyKZZXebmd1O6AF2VwCPBM8qesrdV0Y7QRGp3Pbk5vPAnAye/Ww17ZrU49krhzCsR6tYpyVxKNJZdm5m3wHfEXpgXjNgupnNdfdx0UxQRCqvD7/dwsSZ6WzYsY9Lj+7EzSN60bCObl1LeJHcQ7qB0CPBtwL/BG5297zgabIrABUkEfmB7XsPMPmNZUxflE2XVg146ZpjGJzUPNZpSZyL5FeVlsA57r6m6EZ3LzSzM6KTlohUVm+lb+T2WUvYtvcA153Yld8NVzNUiUwkBemvAGZW9NebXe6ep44NInLQ5l37uWPWEt5a/B192jXm2SsH06edmqFK5CIpSF8CHYFthFoHNQU2mtlm4NfuviiK+YlInHN3pi/KZvIby9iXV8C4ET359c+6qBmqHLJICtLbwCvuPgfAzE4FRgAvEXpW0tDopSci8Wxdzl5ueSWdj1ZsZXBSM+47tz9dWzWMdVpSSUVSkFLc/dqDK+7+jpnd4+43mlmdKOYmInGqsNB57rPV3D8nAwMmjezDxUM7UUPNUOUniKQg5ZjZeGBasH4+sC14RpKeiyRSzWRu3s2EGWmkrtnGCT1acc/ZfenQTM1Q5aeLpCBdCNwBvBqsfxxsSwB+FaW8RCTO5BUUMvXDLB5+dwX16yTwl/OO5JyB7dUMVcpNqQUpOAsa7+6/K2FIZvmnJCLxZvH6HYybnsbSjTv5Zb+23HlWH1o10hV7KV+lFiR3LzCzQRWVjIjEl/15BTz83gqmfphF8wa1eeLiQYzoe0Ss05IqKpJLdl+Z2WzgZWDPwY3uPjNqWYlIzC1cncP46Wlkbd3Dr1I6cOsvkmlSv1as05IqLJIPCjQHvgeGA2cGXxF1aDCzEWaWYWaZZjYhzOt1zOzF4PUvzCypyGsTg+0ZZnZaWTHNrHMQY0UQs3aw/QQz+9LM8s1sdLHjF5jZ18HX7Ejek0hVtzs3nz/OWsx5T3zGgYJC/nXVUO4ffaSKkURdJN2+rzicwMH9p0eBU4BsQo+wmO3uS4sMuwrY5u7dzGwMMAU438ySgTFAH6Ad8K6Z9Qj2KSnmFOAhd59mZk8EsR8H1gKXAzeFSXOfux91OO9PpCqan7GZW2ems3Hnfq44LombTu1JAzVDlQpS5hmSmfUws/fMbHGw3t/Mbosg9hAg092z3P0AoWnjI4uNGQk8GyxPB06y0JSdkcA0d89191WEJk8MKSlmsM/wIAZBzFEA7r7a3dPQFHWREm3bc4AbX/yaK55ZSP06NZl+7bHccWYfFSOpUJFcsnsSmAjkAQQ/3MdEsF97YF2R9exgW9gx7p4P7ABalLJvSdtbANuDGCUdK5y6ZpZqZp+b2agIxotUKe7OG2kbOeWhD5j9zQZuGN6NN244nkGdmsU6NamGIvn1p767Lyj2WYP8kgYXEe7DCcWfNFvSmJK2hyugpY0vS6K7bzCzLsA8M0sv/tBBMxsLjAVITEyMIKRI5bB5535ue3Ux7yzdRL/2TXjuyqEkt2sc67SkGoukIG01s64EP+CDiQEbI9gvm1BT1oM6ABtKGJNtZjWBJkBOGfuG274VaGpmNYOzpHDH+hF33xD8mWVm7wMDgJXFxkwFpgKkpKTo0e1S6bk7L6dmc/cbSzmQX8jE03tx1fGdqalmqBJjkfwLvA74B9DLzNYDfwB+E8F+C4Huwey32oQu8xWfyTab0MP/AEYD89zdg+1jgll4nYHuwIKSYgb7zA9iEMScVVpyZtbsYC8+M2sJHAcsLW0fkcpu7fd7ueSpBYybkUbvto156/c/45phXVWMJC5EMssuCzjZzBoANdx9VySB3T3fzK4H5hBqM/S0uy8xs0lAqrvPBp4CnjezTEJnRmOCfZeY2UuECkQ+cJ27FwCEixkccjwwzcwmA18FsTGzwcArhB67fqaZ3eXufYDewD/MrJBQYb6v2AxAkSqjoND5309X8+c5GSTUMCaP6suFQxLVDFXiioVOLkoZEDqLOBdIokgBc/dJUc0sDqWkpHhqamqs0xA5JCs27WLcjDS+WrudE3u24k9n96Nd03qxTkuqETNb5O4pZY2L5B7SLEKz3xYBuT81MRGpGAfyC3nig5U8Mi+TBnUS+Ov5RzHyqHZqhipxK5KC1MHdR0Q9ExEpN2nZ2xk3PY3l3+3izCPbcceZybRsqGaoEt8iKUifmlk/d0+PejYi8pPszyvgobnf8uRHWbRqVIcnL03hlOQ2sU5LJCKRFKTjgcvNbBWhS3YGuLv3j2pmInJIPs/6ngkz0lj9/V4uGNKRCaf3pkk99Z+TyiOSgnR61LMQkcO2a38e9721nBe+WEti8/r8++qhHNutZazTEjlkJRYkMxvu7vPcfY2ZdQ56yh187RxgTYVkKCIlmrd8E7e+sphNO/dz9fGd+X+n9qRe7YRYpyVyWEo7Q/ozMDBYnlFkGeA2QM9DEomRnD0HmPTaEl79egM92jTksYuOZUCi+s9J5VZaQbISlsOti0gFcHdeS9vInbOXsGt/Hr8/qTvXndiN2jXVaUEqv9IKkpewHG5dRKLsux2hZqjvLtvEkR2aMGX0UHodoWaoUnWUVpC6BE9RtSLLBOudo56ZiAChs6JpC9dxzxvLyCss5NZf9ObK4zuToLY/UsWUVpCKPkzvz8VeK74uIlGw5vs9TJiRzmdZ33N0l+bcd05/klo2iHVaIlFRYkFy9w8qMhER+a+CQueZT1bx53cyqFWjBvee048xgzuq7Y9UaXo+sUicyfgu1Az1m3XbObl3ayaP6scRTerGOi2RqFNBEokTB/ILeez9TB6dn0mjurX42wUDOLN/W50VSbWhgiQSB75et53x09PI2LSLkUe1444z+9C8Qe1YpyVSoUrr1PAapUzvdvezopKRSDWy70ABf3kng6c/WUXrRnV56rIUTuqtZqhSPZXVqUFEouTTlVuZMCOdtTl7uWhoIuNP70XjumqGKtWXZtmJVLCd+/O4981l/GfBOpJa1Gfa2KM5ukuLWKclEnNl3kMys+7AvUAy8H9Tfdy9SxTzEqmS3l26iVtfTWfLrlyuOaELfzi5h5qhigQimdTwDHAH8BBwInAF6mUncki27s7lrteW8to3G+h1RCOevDSF/h2axjotkbgSSUGq5+7vmZm5+xrgTjP7iFCREpFSuDuzvt7AXa8tYXduPjee0oNrh3VVM1SRMCIpSPvNrAawwsyuB9YDraOblkjlt2H7Pm57dTHzlm9mQGJTppzbnx5tGsU6LZG4FUlB+gNQH7gBuBsYDlwWzaREKrPCQuffC9Zy31vLKSh0/nhGMpcdm6RmqCJlKLMgufvCYHE3oftHIlKCVVv3MGFGGl+syuG4bi249+z+JLaoH+u0RCqFSGbZ9QBuBjoVHe/uw6OYl0ilkl9QyFMfr+LBud9Su2YN7j+3P+eldFDbH5FDEMklu5eBJ4AngYLopiNS+SzdsJPxM9JIX7+DU5LbMHlUX9o0VjNUkUMVSUHKd/fHo56JSCWTm1/AI/Myefz9lTStX4tHLxzIL/odobMikcMUSUF6zcx+C7wC5B7c6O45UctKJM4tWrON8TPSyNy8m3MGtOf2M5JppmaoIj9JJB+GuIzQPaRPgUXBV2okwc1shJllmFmmmU0I83odM3sxeP0LM0sq8trEYHuGmZ1WVkwz6xzEWBHErB1sP8HMvjSzfDMbXez4lwXjV5iZZg5KmfYeyOeu15Yw+olP2ZubzzNXDObB849SMRIpB5HMsut8OIHNLAF4FDgFyAYWmtlsd19aZNhVwDZ372ZmY4ApwPlmlgyMAfoA7YB3g8kVlBJzCvCQu08zsyeC2I8Da4HLgZuK5dec0Id7Uwh1NV8UxNp2OO9Xqr6PV2xlwsw0srft49JjOjFuRC8a1tETXETKS5lnSGZWy8xuMLPpwdf1ZhZJS+IhQKa7Z7n7AWAaMLLYmJHAs8HydOAkC12AHwlMc/dcd18FZAbxwsYM9hkexCCIOQrA3Ve7expQWOzYpwFz3T0nKEJzgRERvC+pZnbszWPc9G+4+KkvqJVQg5euOYZJI/uqGImUs0j+Rz0O1AIeC9YvCbZdXcZ+7YF1RdazgaEljXH3fDPbAbQItn9ebN/2wXK4mC2A7e6eH2b8oeT3o33MbCwwFiAxMbGMkFLVvL34O26ftZicPQf4zc+78vuTulO3lpqhikRDJAVpsLsfWWR9npl9E8F+4aYaFX/gX0ljStoe7oyutPGliWgfd58KTAVISUkpK6ZUEVt25XLn7CW8kb6R3m0b8/Rlg+nXoUms0xKp0iIpSAVm1tXdVwKYWRci+zxSNtCxyHoHYEMJY7LNrCbQBMgpY99w27cCTc2sZnCWFO5Y4fL7ebFY75exj1Rx7s7ML9cz6fWl7DtQwM2n9WTsCV2olaBmqCLRFklBuhmYb2ZZhM4qOhFZC6GFQHcz60yoIesY4MJiY2YTmsX3GTAamOfubmazgX+b2YOEJjV0BxYEx/9RzGCf+UGMaUHMWWXkNwe4x8yaBeunAhMjeF9SRa3fvo9bZqbzwbdbGNSpGVPO7U+31g1jnZZItRHJLLv3gof09SRUEJa7e24Zux28J3Q9oR/8CcDT7r7EzCYBqe4+G3gKeN7MMgmdGY0J9l1iZi8BS4F84Dp3LwAIFzM45HhgmplNBr4KYmNmgwl9hqoZcKaZ3eXufdw9x8zuJlQ4ASbps1XVU2Gh868v1jDlreU4cOeZyVx6TBI11AxVpEKZe/jbImY23N3nmdk54V5395lRzSwOpaSkeGpqRB/Bkkpi5ZbdTJiRxsLV2/hZ95bcc3Y/OjZXM1SR8mRmi9w9paxxpZ0hDQPmAWeGec2BaleQpOrIKyjkyY+y+Ou7K6hbswYPjO7P6EFqhioSSyUWJHc/+ETYScFngf5PcA9HpFJavH4H42eksWTDTkb0OYJJo/rQupGaoYrEWiSTGmYAA4ttmw4MKv90RKJnf14Bf5+3gic+yKJZ/do8ftFATu/XNtZpiUigxIJkZr0Ite5pUuw+UmNAv05KpZK6OodxM9LI2rKH0YM6cNsve9O0vvrPicST0s6QegJnAE354X2kXcCvo5mUSHnZk5vPA3MyePaz1bRrUo/nrhzCCT1axTotEQmjtHtIs4BZZnaMu39WgTmJlIsPvt3CLTPT2bBjH5cdk8TNp/WkgfrPicStSP53Xmtmy9x9O0DwQdK/uPuV0U1N5PBs33uAu19fxowvs+nSqgEvX3MMKUnNY52WiJQhkoLU/2AxAnD3bWY2IIo5iRy2t9I3cvusJWzbe4DrT+zG9cO7qRmqSCURSUGqYWbNDj4nKHiOkK57SFzZvHM/f5y1hLeXfEefdo159srB9GmnZqgilUkkheUvwKdmdvBZQ+cBf4peSiKRc3emL8rm7teXsj+/kPEjevHrn3WmppqhilQ6kfSye87MFgEnEupld06xp76KxMS6nL3c8ko6H63YyuCkZtx3bn+6tlIzVJHKKqJLb0Gz0y0Enz8ys0R3XxvVzERKUFDoPPfZah6Yk4EBd4/sw0VDO6kZqkglV2ZBMrOzCF22awdsJvT4iWXoI75QAAARBElEQVSEPjQrUqEyN+9i/Ix0Fq3ZxrAerfjT2X3p0EzNUEWqgkjOkO4GjgbedfcBZnYicEF00xL5obyCQv7xwUr+9l4m9esk8OCvjuTsAe3VDFWkComkIOW5+/dmVsPMarj7fDObEvXMRAKL1+/g5ulpLNu4k1/2b8udZ/ahVaM6sU5LRMpZJAVpu5k1BD4EXjCzzYQemicSVfvzCvjruyt48qMsmjeozT8uGcRpfY6IdVoiEiWRFKSRwD7gf4CLgCbApGgmJbJgVQ4TZqSRtXUP56d05JZf9KZJ/VqxTktEoqjUgmRmCcAsdz8ZKASerZCspNratT+P+9/O4PnP19ChWT3+ddVQju/eMtZpiUgFKLUguXuBme01sybuvqOikpLqaX7GZm6dmc7Gnfu58rjO3HRaD+rXVlMQkeoikv/t+4F0M5sL7Dm40d1viFpWUq1s23OAu19fysyv1tO9dUOmX3ssgzo1i3VaIlLBIilIbwRfIuXK3XkjfSN3zFrCjn153DC8G9cN70admmqGKlIdlfbE2ER3X+vuum8k5W7Tzv3c9upi5i7dRL/2TfjX1UPp3bZxrNMSkRgq7QzpVWAggJnNcPdzKyYlqcrcnZdS1zH5jWUcyC9k4um9uOp4NUMVkdILUtGPwHeJdiJS9a39fi8TZqbx6crvGdK5OVPO7U/nlg1inZaIxInSCpKXsCxySAoKnf/9dDV/npNBQg1j8qi+XDgkUc1QReQHSitIR5rZTkJnSvWCZYJ1d3dd8JcyfbtpF+Omp/H1uu0M79WayaP60q5pvVinJSJxqMSC5O6a6iSH7UB+IU98sJK/z1tBwzo1eXjMUZx1ZDs1QxWREkX1TrKZjTCzDDPLNLMJYV6vY2YvBq9/YWZJRV6bGGzPMLPTyoppZp2DGCuCmLVLO4aZJZnZPjP7Ovh6Inrfierlm3XbOeuRj3lw7reM6NuWd28cxsij1JlbREoXtYIUtB16FDgdSAYuMLPkYsOuAra5ezfgIWBKsG8yMIbQM5dGAI+ZWUIZMacAD7l7d2BbELvEYwRWuvtRwde15fj2q6V9Bwq4581lnP3YJ2zbe4AnL03h7xcMoEVDdeYWkbJF8wxpCJDp7lnufgCYRqhRa1Ej+W9/vOnASRb6NXokMM3dc919FZAZxAsbM9hneBCDIOaoMo4h5eizld9z+sMfMvXDLM4f3JG5Nw7jlOQ2sU5LRCqRaDYKaw+sK7KeDQwtaYy755vZDqBFsP3zYvu2D5bDxWwBbHf3/DDjSzoGQGcz+wrYCdzm7h8dxvus1nbuz+O+t5bz7y/Wkti8Pv++eijHdlMzVBE5dNEsSOHOQopPHy9pTEnbw53RlTa+tGNsBBKDhw8OAl41sz7uvrPoQDMbC4wFSExMDBOq+pq3fBO3zFzM5l37+fXPOnPjKT2pV1tzYUTk8ESzIGUDHYusdwA2lDAm28xqEnrWUk4Z+4bbvhVoamY1g7OkouPDHsPdHcgFcPdFZrYS6AGkFk3Q3acCUwFSUlL0eSzg+925THp9KbO+3kDPNo144pJBHNWxaazTEpFKLpr3kBYC3YPZb7UJTVKYXWzMbOCyYHk0MC8oFLOBMcEMuc5Ad2BBSTGDfeYHMQhizirtGGbWKpgkgZl1CY6RVY7vv8pxd2Z9vZ5THvqQN9M38oeTu/Pa745XMRKRchG1M6Tgfs31wBwgAXja3ZeY2SQg1d1nA08Bz5tZJqEzozHBvkvM7CVgKaHHpV/n7gUA4WIGhxwPTDOzycBXQWxKOgZwAjDJzPKBAuBad8+J1vejstu4Yx+3vbKY95Zv5siOTbn/3P70PKJRrNMSkSrEQicXEomUlBRPTU0te2AVUljoTFu4jnvfXEZeYSE3ndqTK47rTILa/ohIhMxskbunlDVOj+OUEq3euocJM9P4PCuHY7q04L5z+9GphZqhikh0qCDJjxQUOk9/vIq/zM2gVo0a3HdOP84f3FGdFkQkqlSQ5AeWf7eT8dPT+CZ7Byf3bs3kUf04okndWKclItWACpIAkJtfwKPzV/LY/Eya1KvF3y8YwBn92+qsSEQqjAqS8NXabYyfkca3m3Yz6qh2/PHMPjRvUDvWaYlINaOCVI3tPZDPX975lqc/WcURjevy9OUpDO+l/nMiEhsqSNXUp5lbmTAznbU5e7loaCITTu9Fo7q1Yp2WiFRjKkjVzI59edz75jKmLVxHUov6TBt7NEd3aVH2jiIiUaaCVI3MXbqJ215NZ8uuXK4Z1oX/ObkHdWupGaqIxAcVpGpg6+5c7py9hNfTNtLriEY8eWkK/Tuo/5yIxBcVpCrM3Xn16/Xc9dpS9uYW8P9O6cE1w7pSu2ZUn1wvInJYVJCqqA3b93HrK+nMz9jCgMRQM9TubdQMVUTilwpSFVNY6LywYC1T3lpOQaHzxzOSuezYJDVDFZG4p4JUhWRt2c2EmeksWJXD8d1acu85/ejYvH6s0xIRiYgKUhWQX1DIPz9exUNzv6V2zRrcf25/zkvpoLY/IlKpqCBVcks37GTcjG9YvH4npya34e5RfWnTWM1QRaTyUUGqpHLzC3hkXiaPv7+SpvVr8eiFA/lFvyN0ViQilZYKUiW0aE2oGWrm5t2cM7A9t/8ymWZqhioilZwKUiWyJzefP7+Twf9+upp2Terxv1cM5uc9W8c6LRGRcqGCVEl8tGILE2emk71tH5ce04lxI3rRsI7++kSk6tBPtDi3Y28ek99YysuLsunSsgEvXXMMQzo3j3VaIiLlTgUpjr29+Dtun7WYnD0H+M3Pu/L7k7qrGaqIVFkqSHFo86793Dl7CW+mf0dy28Y8c/lg+rZvEuu0RESiSgUpjrg7M79cz6TXl7Ivr4CbT+vJ2BO6UCtBzVBFpOpTQYoT2dv2cssri/nw2y0M6tSMKef2p1vrhrFOS0SkwqggxVhhofP852uY8vZyAO46qw+XHN2JGmqGKiLVjApSDK3cspvx09NIXbONn3VvyT1nqxmqiFRfUb05YWYjzCzDzDLNbEKY1+uY2YvB61+YWVKR1yYG2zPM7LSyYppZ5yDGiiBm7cM9RrTlFRTy6PxMTn/4I1Zs3s2fzzuS564comIkItVa1AqSmSUAjwKnA8nABWaWXGzYVcA2d+8GPARMCfZNBsYAfYARwGNmllBGzCnAQ+7eHdgWxD7kY5Tvd+HHFq/fwahHP+GBORmc1Ks1c288gdGD1JlbRCSaZ0hDgEx3z3L3A8A0YGSxMSOBZ4Pl6cBJFvrJPBKY5u657r4KyAzihY0Z7DM8iEEQc9RhHiMq9ucVcP/byxn56Cds2pnL4xcN5PGLB9G6kTpzi4hAdO8htQfWFVnPBoaWNMbd881sB9Ai2P55sX3bB8vhYrYAtrt7fpjxh3OMcrUuZy+XPbOArC17OG9QB277ZTJN6teKxqFERCqtaBakcNegPMIxJW0Pd0ZX2vjDOcYPEzQbC4wFSExMDLNL2do0rktSiwbceWYfTujR6rBiiIhUddG8ZJcNdCyy3gHYUNIYM6sJNAFyStm3pO1bgaZBjOLHOtRj/IC7T3X3FHdPadXq8IpJ7Zo1ePrywSpGIiKliGZBWgh0D2a/1SY0gWB2sTGzgcuC5dHAPHf3YPuYYIZcZ6A7sKCkmME+84MYBDFnHeYxREQkBqJ2yS64X3M9MAdIAJ529yVmNglIdffZwFPA82aWSeisZUyw7xIzewlYCuQD17l7AUC4mMEhxwPTzGwy8FUQm8M5hoiIVDwLnSxIJFJSUjw1NTXWaYiIVCpmtsjdU8oap66dIiISF1SQREQkLqggiYhIXFBBEhGRuKCCJCIicUGz7A6BmW0B1vyEEC0JfYi3OtF7rh70nquHw33Pndy9zM4AKkgVyMxSI5n6WJXoPVcPes/VQ7Tfsy7ZiYhIXFBBEhGRuKCCVLGmxjqBGNB7rh70nquHqL5n3UMSEZG4oDMkERGJCypIFcDMRphZhpllmtmEWOdTEczsaTPbbGaLY51LRTCzjmY238yWmdkSM/t9rHOqCGZW18wWmNk3wfu+K9Y5VQQzSzCzr8zs9VjnUlHMbLWZpZvZ12YWlS7TumQXZWaWAHwLnELooYALgQvcfWlME4syMzsB2A085+59Y51PtJlZW6Ctu39pZo2ARcCoavD3bEADd99tZrWAj4Hfu/vnMU4tqszsRiAFaOzuZ8Q6n4pgZquBFHeP2mevdIYUfUOATHfPcvcDwDRgZIxzijp3/5DQ86eqBXff6O5fBsu7gGVA+9hmFX0esjtYrRV8Venfcs2sA/BL4J+xzqWqUUGKvvbAuiLr2VSDH1TVmZklAQOAL2KbScUILl99DWwG5rp7VX/ffwXGAYWxTqSCOfCOmS0ys7HROIAKUvRZmG1V+jfI6szMGgIzgD+4+85Y51MR3L3A3Y8COgBDzKzKXqI1szOAze6+KNa5xMBx7j4QOB24LrgsX65UkKIvG+hYZL0DsCFGuUgUBfdQZgAvuPvMWOdT0dx9O/A+MCLGqUTTccBZwf2UacBwM/tXbFOqGO6+IfhzM/AKodsR5UoFKfoWAt3NrLOZ1QbGALNjnJOUs+Dm/lPAMnd/MNb5VBQza2VmTYPlesDJwPLYZhU97j7R3Tu4exKh/8vz3P3iGKcVdWbWIJisg5k1AE4Fyn0GrQpSlLl7PnA9MIfQje6X3H1JbLOKPjP7D/AZ0NPMss3sqljnFGXHAZcQ+o356+DrF7FOqgK0BeabWRqhX77munu1mQpdjbQBPjazb4AFwBvu/nZ5H0TTvkVEJC7oDElEROKCCpKIiMQFFSQREYkLKkgiIhIXVJBERCQuqCCJlCMza2pmvy2y3s7MpkfpWKPM7I/RiF3GcWub2YdmVrOijy1Vm6Z9i5SjoI/d6xXR4dzMPgXOilb3ZTOrGXyOLtxrdxBqGvxCNI4t1ZPOkETK131A1+CDsQ+YWdLBZ0KZ2eVm9qqZvWZmq8zsejO7MXiuzudm1jwY19XM3g6aWH5kZr2KH8TMegC57r7VzBoF8WoFrzUOnl1Tq6RYZnammX0RHPtdM2sTbL/TzKaa2TvAc2bWJ3je0ddmlmZm3YMUXgUuivp3U6oVFSSR8jUBWOnuR7n7zWFe7wtcSKgP2J+Ave4+gFBXi0uDMVOB37n7IOAm4LEwcY4Dij7u4n1Cj0SAUEubGe6eV0qsj4Gjg2NPI9S9+qBBwEh3vxC4Fng4aJ6aQqg3I4TaxgyO6DsiEiFdAxapWPODArLLzHYArwXb04H+QbfwY4GXQ+3xAKgTJk5bYEuR9X8SKiqvAlcAvy4jVgfgxeDBgrWBVUVizXb3fcHyZ8CtwTOAZrr7Cgh1+DazA2bWKHg/Ij+ZzpBEKlZukeXCIuuFhH5BrAFsD86wDn71DhNnH1D34Iq7fwIkmdkwIMHdF5cR6+/AI+7eD7imaCxgT5G4/wbOCo43x8yGFxlXB9h/qN8AkZKoIImUr11Ao8PdOXiG0iozOw9CXcTN7MgwQ5cB3Yptew74D/BMBLGaAOuD5ctKysfMugBZ7v43Ql3q+wfbWwBbgsuCIuVCBUmkHLn798AnZrbYzB44zDAXAVcFnZWXEP6R9x8CA6zItTjgBaAZoaJUVqw7CV3K+wgobZbe+cDi4ImwvQgVPYATgTcP5U2JlEXTvkUqKTN7GHjN3d8N1kcTmoxwSQUceyYw0d0zon0sqT40qUGk8roHGApgZn8n9GjpqD+DKXjQ5KsqRlLedIYkIiJxQfeQREQkLqggiYhIXFBBEhGRuKCCJCIicUEFSURE4oIKkoiIxIX/Dxh3Xmb2CQK0AAAAAElFTkSuQmCC\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "nbodies = 2\n",
    "x = np.zeros([nbodies,2])\n",
    "v = np.zeros([nbodies,2])\n",
    "m = np.zeros([nbodies])\n",
    "SetSunIC(x,v,m)\n",
    "SetEarthIC(x,v,m)\n",
    "\n",
    "x_results = np.zeros([nsteps,nbodies,2])\n",
    "E_results = np.zeros([nsteps,3])\n",
    "\n",
    "xv = np.zeros([nbodies,4]) # contains both x's and v's for RK4\n",
    "xv[:,0:2] = x\n",
    "xv[:,2:4] = v\n",
    "t = 0\n",
    "for i in range(nsteps):\n",
    "    xv = rk4step(dxdt_dvdt, t, dt, xv, m)\n",
    "    t = t + dt\n",
    "    x_results[i] = xv[:,0:2]\n",
    "    E_results[i] = CalculateEnergy(xv[:,0:2], xv[:,2:4], m)\n",
    "    \n",
    "plot.plot(x_results[:,0,0], x_results[:,0,1], label='sun')\n",
    "plot.plot(x_results[:,1,0], x_results[:,1,1], label='earth')\n",
    "plot.axis('equal')\n",
    "plot.show()\n",
    "\n",
    "time = np.linspace(0.0, tfinal, num=nsteps, endpoint=False)/(2.0*math.pi) # in years\n",
    "plot.plot(time, E_results[:,0], label='Kinetic energy')\n",
    "plot.plot(time, E_results[:,1], label='Potential energy')\n",
    "plot.plot(time, E_results[:,2], label='Total energy')\n",
    "plot.legend()\n",
    "plot.xlabel('time (years)')\n",
    "plot.ylabel('Energy')\n",
    "plot.show()\n",
    "plot.plot(time, (E_results[:,2]-E_results[0,2])/E_results[0,2], label='Total energy change')\n",
    "plot.xlabel('time (years)')\n",
    "plot.ylabel('Fractional Energy error')\n",
    "plot.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "The error is even smaller!  But note that the amplitude of the error is growing with time, whereas the leapfrog error just keeps oscillating.  In fact, if we increased the timestep, the RK4 integrator would rapidly do much worse than the leapfrog.  This is indicative of the fact that the leapfrog has a special property -- is is symplectic and so the energy error will stay bounded.  On the other hand, if we decreased dt further, the error on the RK4 integrator drops rapidly since it is fourth-order; whereas the leapfrog is only second-order."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Problem 2 - Add in Jupiter-like orbit"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Set up Jupiter-like object in eccentric orbit at pericenter.  First, repeat the steps from above to create the necessary arrays for 3 bodies."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 20,
   "metadata": {},
   "outputs": [],
   "source": [
    "nbodies = 3\n",
    "x = np.zeros([nbodies,2])\n",
    "v = np.zeros([nbodies,2])\n",
    "m = np.zeros([nbodies])\n",
    "SetSunIC(x,v,m)\n",
    "SetEarthIC(x,v,m)\n",
    "\n",
    "tfinal = 200.0*torbit  # set number of years (earth orbits) to run\n",
    "nsteps = int(tfinal/dt)  # number of steps\n",
    "x_results = np.zeros([nsteps,nbodies,2])\n",
    "E_results = np.zeros([nsteps,3])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Now, added in Jupiter.  Results may vary because I didn't specify the phase of the orbit."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "metadata": {},
   "outputs": [],
   "source": [
    "semi_major = 4*AU\n",
    "eccentricity = 0.6\n",
    "vorbit = math.sqrt(G*mass_solar/semi_major*(1+eccentricity)/(1-eccentricity))\n",
    "x[2] = np.array([(1-eccentricity)*semi_major, 0.0])\n",
    "v[2] = np.array([0.0, vorbit])\n",
    "m[2] = mass_jupiter"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Adjust solar velocity to make center-of-mass velocity zero.  This is not entirely necessary but is useful because with the addition of jupiter, the system tends to drift (since we are in a reference frame in which the center-of-mass motion is non-zero)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 22,
   "metadata": {},
   "outputs": [],
   "source": [
    "v[0] += np.array([0.0, -vorbit*mass_jupiter/mass_solar])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Now do the integration"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 23,
   "metadata": {},
   "outputs": [],
   "source": [
    "a = ComputeAccel(x, m)\n",
    "v = v + 0.5*dt*a  # Take a half-step to start the system\n",
    "for i in range(nsteps):\n",
    "    (x, v) = LeapFrog(x, v, m, dt)\n",
    "    x_results[i] = x\n",
    "    E_results[i] = CalculateEnergy(x, v, m)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Plot orbit results"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 24,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAXYAAAD8CAYAAABjAo9vAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADl0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uIDIuMi4zLCBodHRwOi8vbWF0cGxvdGxpYi5vcmcvIxREBQAAIABJREFUeJzt3Xd8VGW6B/DfkwAJEECqUjcooNKkBBTLIggsAi4u6ipXFEQXwUVFxQWxt9Vrr6jcq6IuKoiyuiqryLWxIpIgIhhABJUIQoiClBBS3vvHmzC9ZU4/v+/nM5/MKXPOM5mZZ955z1tEKQUiIvKODLsDICIiYzGxExF5DBM7EZHHMLETEXkMEzsRkccwsRMReQwTOxGRxzCxExF5DBM7EZHH1LHjpC1atFC5ubl2nJqIyLUKCgp2KaVaJtrPlsSem5uL/Px8O05NRORaIvJDMvuxKoaIyGOY2ImIPIaJnYjIY5jYiYg8homdiMhjmNiJiDyGiZ2IyGNsacdOZLoDB4DiYuD774EffwS++w4oLNT3t2wBduyI//iMDODoo4FOnfTf444DjjkG6NABaNMGaNIEyMy05KkQpYqJndxn1y7g88+BRYuA554z5xxVVcCmTfqWig4dgCuuAEaNAo49FqjDjxhZT+yYzDovL0+x5ynFpRTw7bfAggXAzTfbHY0x2rYFHnkEGDECaNDA7mjIhUSkQCmVl2g/1rGT/aqqdDXJ5ZcDIvqWkaFLvF5J6gDw00/AeecBDRsGnqcI8OCDwO7ddkdHHsLETtarqADefVcnbxFdV921KzBnjt2R2WP6dKBp00Ci79EDWL/e7qjIxVgBSOZTSteJjxhhT8l00CCgb1+gZ0+gXTvgqKOARo10ybl+faBePb1fTWKtibm8HCgt1bf9+3Xd/vbt+tfFl18C778P7NljfLxr1wLHHx9YPvlk4PXXddxESWAdO5lj/37goYeAW24x7xxt2wJTpwLDh+vWKzk55p0rFZWVwLZtwKefAv/7v8CHHxp7/JdeAsaOZascH0q2jj3txC4i2QA+AZAF/QtgoVLq1niPYWL3qF9/BSZMAN56y9jjDh4MXH898Pvfu/+io1K66eUzzwAPPJD+8a64Qn+BZmWlfyxyPCsvnpYBGKyUOgFALwDDReQkA45LbvDbb8D48boKo1mz9JP6kCHAypW61KuUvi1dqkvlbk/qgP4/deoE3H9/4PkpBZSUAHfemfrxZs8GsrP1cS+8UFcfke+lndiVtq96sW71zfr6HbJOVRXw+OM6mTRpArz4Yu2PNWOG7ixUk+CWLAHy8vSFVT9p1gy46abA/6GyEvjoo9SO8fLL+nqBCPD00/o45EuGfHpEJFNEVgPYCWCJUmqFEcclh9m0KdCK5aqraneMiy/W9c81Cezee4FWrYyN0wsyMoCBA0MT/YIFyT9+ypRAq6OtW82LkxzJkMSulKpUSvUC0A5AfxHpHr6PiEwSkXwRyS8uLjbitGQFpXSnGhGgc+faHeOf/wxUrbzwAtC6tbEx+kFGhm4DX5Pod+0CBgxI7rEdOujXb+ZMluJ9wvBWMSJyK4D9SqmYV4Z48dQFDhwAhg4FPvusdo9fsQLo1y/QfJDMU1amO3e98ELyjzlwQDf1JFex7OKpiLQUkSOq79cHMAQAe1e41Y4dOhk3bJh6Ul++PFCi7N+fSd0qWVnA3Ln6/15WplsRJdKggX59vvvO9PDIekZUxbQG8KGIrAGwErqO/W0DjktW2rpVf9BT7QQze7buSaoUcBIbQ9muXj3dikgp4OefE+/fqZN+3VetMj82skzaPU+VUmsA9DYgFrLDDz8AubmpPaZjRyA/X7fkIOc68shAnfqzzwKXXRZ737599d+PPtIXbcnVfNamjA775RddUkslqT/1lG7quHkzk7rbXHpp4KJrPKefzhK8BzCx+83Bg7ok17x58o9Zs0YnhcmTWW/uds2bB5pP9usXe7++ffVr/dNP1sVGhmFi9wulgCuv1C0hdu5M7jE1HYd69DA3NrJeRgbwxRf69b3nntj7tWunE/yBA9bFRmljYveDzz7TH+Qnnki8b1aWHrFQKXYc8oua9u3PPht7n4YN9fSA5ApM7F62d68ubZ1ySuJ9MzL0/gcPAo0bmx8bOc/EiYFOZNFs2KDfT0YP8kaGY2L3qrvuSj5B79ql61ydMuwt2evii3WCvzXGIK2jR+sEv3+/tXFR0pjYvaakRH/okplSbvNm/QFO5UIq+cdtt+lWULGqYHJy9DDN5DhM7F5y881AixaJ9/vXv3RC79jR/JjI3UT0jFGHDkXf/sILep+SEmvjoriY2L3gwAH94brrrvj7jR6tq1xGjbImLvKOunV1YWDDhujbW7TQY+aTIzCxu9177+kWC4mUlOhRFv02zjkZq0sXneAvuSRy23vv6QJGrNI9WYafcrdSCjj22MSlpJrBodhTlIz03HO6BVU0WVmpjR1PhmNid6N9+3TJe+PG+PuVlupp64jMkJWlCw1PPhm57fzz2UvZRkzsblNQADRqFH+fV1/VH7jsbGtiIn+74orY1S8iuoBBlmJid5M77tDzgcazd68uLRFZqebi6syZkdsaNACWLbM+Jh9jYneLtm1jdxgBdKlJKXYyInvdc48eOTTcaaclNwEIGSLt8djJZJWVQJ0EL9PatUC3btbEQ5RI06a6kBFex/7hh3od5101HUvsTlZamjipHzzIpE7OpBTw0EOR60V0j1YyDRO7U/36q66bjGXIEP3BycqyLiaiVF1zDbBtW+T6zMzYzSUpbUzsTrR9e/x25y+/DCxZYl08ROlo3VpXKYarXx8oKrI+Hh9gYneaLVuANm1ib1+/Hhg71rp4iIyQkRG9br19ez1/LhmKid1JNm8Gjj469vbfftO9TYncSimgf//Qdf366Um0yTBpJ3YRaS8iH4pIoYisE5GrjQjMd7ZuBY45Jvb28vLEHZOI3GDFCuDBB0PXDRoEfPCBPfF4kBEl9goA1ymljgdwEoC/ikhXA47rHzt2AB06xN5eVZW4dQyRm1x7LfDOO6Hrhg4FFi+2Jx6PSTuxK6W2K6VWVd/fC6AQQNt0j+sb+/YBRx0Ve3u09sBEXjBiBPDVV5HrmNzTZmgdu4jkAugNYIWRx/WsiorY1StHHMGOHOR9PXvqashgI0YAq1fbE49HGJbYRSQHwOsApimlfouyfZKI5ItIfnFxsVGndS+l9Pga0TRtqtuxE/lBu3aRbd1799Zz8VKtGJLYRaQudFKfp5R6I9o+Sqk5Sqk8pVRey5YtjTitu8Wa8CI7O/pYG0Re1ro18MMPoetatuSkHbVkRKsYAfAsgEKlVJT+wxThnHNib+MQp+RXHTrocY+C1Yz5TikxosR+CoCLAAwWkdXVtxEGHNeb3n4beCPqjxqOn0HUrRvw8ceh6zidY8rSbkOnlFoGgM02krFrF3DWWdG3VVay9QsRAPz+98CcOcCkSYF1HBUyJfwqtIpSus4wmkOHWCohCvaXvwADB4auizaBNkXFbGKVWIm7uDh26xgiPwsfZmDuXGDTJjsicR0mdis8/3z09cuXAy1aWBsLkZuEV7907sxrUUlgYjfbvn3AxImR62+5BTjpJOvjIXKb8ESemWlPHC7CxG62WD1Lb7/d2jiI3Eoksm8HGxrExcRupmgztgP8KUmUqqZNgdmzQ9cVFtoTiwuIsqEJUV5ensr3+uD6+/ZFL63v3Bm7dQwRxRdeUvdZE0gRKVBK5SXajyV2s0RL6rNnM6kTpSM8kbNKJiomdjMsWhR9/ZQp1sZB5EW7d4cue/3Xfy0wsRtNKWDMmMj15eXWx0LkRU2aACecEFju18++WByKid1oXaNMHvX++5wBichI4eO1s0omBBO7kcrKgPXrI9cPHWp9LEReFz4SKkdGPYyJ3UjZ2ZHrysqsj4PID8I/bw0a2BOHAzGxGyXa5Bj33gvUq2d9LER+Ed5K5sMP7YnDYZjYjdK8eeS6GTOsj4PIbwoKAvcHD7YvDgdhYjdCtPlJN2ywPg4iP+rTJ3R58mR74nAQJnYjNGsWua5LF+vjIPKr4GtZzzxjXxwOwcSeroMHI9ft3Wt9HER+Fn4t67TT7InDIZjY01W/fuS6nBzr4yDyu+DB9ZYtsy8OB2BiT0e0AYiileCJyHzhnZSizYPgE0zs6bjmmsh1WVnWx0FEWnBhK9bMZT7AxJ6ORx8NXWZpnchZli61OwJbGJLYReQ5EdkpImuNOJ4r7NwZuY6ldSL7Bde1DxliXxw2MqrEPhfAcIOO5Q5HHhm6vGOHPXEQUajwuvaKCnvisJEhiV0p9QmAKH3qfaRVK7sjIDLMqFE6P9bmNnWq3dFDz2BWo25d++KwiWV17CIySUTyRSS/uLjYqtOa44svQpf/9S974iAyiFKhyfmdd2p/rCefDD2WLRo2tOnEzmBZYldKzVFK5Sml8lq6fXq4E08MXR41yp44iNI0f75OvhkmZoKaBP/66+adI6o77gjcf/ddi09uL7aKIfKhJUt0sr3ggsT7Dh+uq6mVin6rqgJefTXxcc49V59z+fL040/KzTcH7o8cadFJnYGJPVVFRaHLv/1mTxxEtVBRoZPrsGGx93n00dDEvXgxkJkZe38R4PzzQx/z44+x9z/5ZP2YaP37yBhGNXd8BcByAMeKSJGIXGrEcR2pffvQ5UaN7ImDKEUi8a8j1iTlq65K/1zt2weO9/vfR98nI8OCOvjg1mr332/yyZxDlA1fm3l5eSrfrTOLB78Te/cGVq2yLxYPKS8vR1FREQ6ykxcAIDs7G+3atUNdg1p0xEugVqWAMWOARYtsiCH4ybv8Z4KIFCil8hLtxxmW0/H553ZH4BlFRUVo1KgRcnNzIT6fmFgphZKSEhQVFaFjx45pHauyMvY86lbnuDfe0H+jvbysmjEW69hTsWRJ6DKnvTPMwYMH0bx5c98ndQAQETRv3jztXy/l5dGT+qRJ9iZRpSIvVQEmJvfgXuLff2/CCZyHJfZUxLviRGljUg9I939RWhp9bueqKhvblgdp2zbQdj5YRoa+wBvvYm3KgptXd+zoi58GLLHX1tatdkdAFJVS0ZN6tERqt2g5NlbVESWPib222rWzOwKiqKJ1NnJyITVabIZ/Af3Xfxl8QGdjYk9WZaXdEZDJ9u/fj5EjR+KEE05A9+7dMX/+fOTm5mLXrl0AgPz8fJx++ukAgNtuuw0TJ07E6aefjqOPPhqPPfaYjZEHREuITk7qNUxP7vPmBe6vXm3ggZ2JiT1ZDzwQuN+kiX1xkGn+/e9/o02bNvjqq6+wdu1aDB8ef8DS9evX47333sMXX3yB22+/HeXl5RZFGt3MmZHr3JDUa0SLddo0E07Uu7cJB3UW1mYlK/hTs3GjfXH4wO3/Wodvthnbo7drm8a49axucffp0aMHpk+fjhkzZmDUqFE4LcGEyCNHjkRWVhaysrLQqlUr7NixA+1srKL77/8OXQ4e4NAtwq8DPPoo8Mgj9sXjVkzstcEhej2pS5cuKCgowLvvvosbbrgBw4YNQ506dVBVPXFDePPDrKCJVTIzM1Fh47jf0aot3DrA4fbtQOvWgWW2cU8dEzs5TqKStVm2bduGZs2aYdy4ccjJycHcuXORm5uLgoICnHnmmXjd8uEJk7N/f+Q6NyfCo46KXFdZaUATyIoK3zS58cezJErC119/jeuvvx4ZGRmoW7cunnrqKZSWluLSSy/F3//+d5wYPlyzQ+TkhC57YVSG8CqZOnUM+LIK/mYoKvJ0yzaOFZMsD4034USFhYU4/vjj7Q7DUZL5n1RVRZZkvfL2zMkJ/TViSOcql3+Okx0rhq1iUrVund0REB3m1aQORF78NXMyEK/hvyoZ27cH7rNUSUQOx8SejKuvDtx3Wp9s8q1ly0KXt22zJw4zhf8C+eore+JwGyb2ZLz2mt0REEUIb2Yf3ETQq3r1SvMApaWGxOF0TOxE5GiTJhl4sOxsAw/mXEzsRC5U3WfqsMJCe+KwwjPP2B2B+zCxExlk7ty52BZU0R08gJjRwpPdcceZchpHmjvX7gicj4mdyACVlZURid1MV1xhyWkc6ZJLDDpQ+M8eD2FiJwryj3/8A/3790evXr1w+eWXo7KyElOmTEFeXh66deuGW2+99fC+ubm5uOOOO3DqqafilVdeQX5+Pi688EL06tULpdUX6R5//HH06dMHPXr0wPr16+16WhTN5s12R2AaQxK7iAwXkQ0isklEogweSuR8hYWFmD9/Pv7zn/9g9erVyMzMxLx583D33XcjPz8fa9aswccff4w1a9Ycfkx2djaWLVuGcePGIS8vD/PmzcPq1atRv359AECLFi2watUqTJkyBQ8ED/1MKVm61ISD/v3vJhzUGdIeK0ZEMgE8CWAogCIAK0XkLaXUN+kem/xp2jTj50Lo1Svx8K9Lly5FQUEB+vXrBwAoLS1Fq1atsGDBAsyZMwcVFRXYvn07vvnmG/Ts2RMAcP7558c95pgxYwAAffv2xRtvvJH+E/GpQYNMOOjzzwPPPWfCge1nxCBg/QFsUkptBgAReRXAaABM7OQqSimMHz8e99xzz+F1W7ZswdChQ7Fy5Uo0bdoUEyZMCBm+t2GCsXFrhvY1c1jfaKMhmil35jsAgO/vHWnZOdkvMDVGJPa2AIJndi4C4Mxh8MgV7JpY4YwzzsDo0aNxzTXXoFWrVvjll1/w448/omHDhmjSpAl27NiBxYsXH54eL1yjRo2wd+9ea4MGMG6cdefa+ZsHho70ASPq2KN9l0YMRSQik0QkX0Tyi4uLDTgtkbG6du2Ku+66C8OGDUPPnj0xdOhQZGVloXfv3ujWrRsmTpyIU045JebjJ0yYgMmTJ4dcPLXC/PmWnQqtGvujg4/bpT1sr4gMAHCbUuoP1cs3AIBS6p5Yj3HdsL0uH+rTDThsb6R4/5Pwqgmvvy2VCh3dMa3nW/PPu/Za4MEH04rLalYO27sSQGcR6Sgi9QBcAOAtA45LRAQA+OknEw56550mHNQZ0q5jV0pViMhUAO8ByATwnFKKg5YTkWHatzfhoA0amHBQZzBkajyl1LsA3jXiWERElB72PE3GwIF2R0AUomNHuyMgJ2NiT8YTTwTur11rXxxE1cJHJ/D6xdNgHh7ixTBM7Mno1i1wP3x2AyIb1KsXuuzl+UBXrQpdTquzkk++FTz8djBQ8Dtp92774iDTnXzyybV63NNPP40XX3wRQOTwvZSevn0NPJgpV2Gdx5CLp0Re8dlnn9XqcZMnTz58f+7cuejevTvatGmT9OMrKipQp05qH8fjjgutklHK+13vU/wXRfLJFy5L7ERBcnJy8NFHH2HUqFGH102dOhVzq2d3yM3NxYwZM9C/f3/0798fmzZtAgDcdttteOCBB7Bw4cKI4XsLCgowcOBA9O3bF3/4wx+wfft2AMDpp5+OWbNmYeDAgXj00UdTjjV81iQvVsdcc03ocnm5PXG4jQffChbYt8/uCMhGjRs3xhdffIGpU6di2rRpIdvOPffckOF769SpgyuvvBILFy5EQUEBJk6ciBtvvPHw/rt378bHH3+M6667zuqn4Qp2jRvkdqyKqY1zzgHee8/uKLzLrnF7kzR27NjDf68JL1KG2bBhA9auXYuhQ4cC0DMttW7d+vD2RMP+JvLVV8AJJwSWRbzTQib8F8mGDQYe3OPXypjYk/Xaa8B55+n7779vbyxkqjp16qAqqPVE8DC9ACBBFdmSoFJbKYVu3bph+fLlUbcnGvY3keph4T2pa9fQ5S5d0jxg8DdekyZpHszZmNiTdc45dkfgHzb//v7d736Hb775BmVlZTh48CCWLl2KU0899fD2+fPnY+bMmZg/fz4GDBgQ8fjg4XuPPfZYFBcXY/ny5RgwYADKy8uxceNGdAtuQpumuXOBCRMCy14otYd/X86ZY8BBvXgRIgYm9mSFv9MOHPD0WBN+JSJo3749/vznP6Nnz57o3LkzevfuHbJPWVkZTjzxRFRVVeGVV16JOEbN8L3169fH8uXLsXDhQlx11VXYs2cPKioqMG3aNEMT+/jxoYkdANatC+1+4SbRvpT+8hfr43CztIftrQ3XDdtbIzi59+0LuPE5OJQThu0tKSlBnz598MMPP8TcJzc3F/n5+WjRooXp8aTyP9mzBzjiiNB1bi21h5ehDh4EqieiMu7ALv3nWDlsrz8VFNgdARlo27ZtGDBgAKZPn253KLUSrcrYjW3ao8VsSFIPVt3c1MtYFZOKLVtCR1/yQ48Qn2jTpg02btyYcL/vv//e/GBqKdrb0U317WeeGbnOsNivvjpw3+pJYm3AEnsqcnNDl2vRqYTITNHmy3ZD2eO++4B//zt0naHDujz2mIEHcz4m9nQkaMNMqbHjeo9T1fZ/kZkJfPBB5HonJ/fJk4EZM0LX/d//OTtmp2NiT1VYCwm/jBZntuzsbJSUlDC5Qyf1kpISZGfXbuLoM84AhgyJXO/ERCkCPPNM6LquXYFBgww8yZ49gfvVzVC9jq1iUlVWBgR/4KZNAx5+2L54PKK8vBxFRUURnYH8Kjs7G+3atUPdunXTOIZ+u4ZzyndnrC8aw+PzQGuYGsm2imFirw2/TRFPrjV9OvDgg5Hr//Qn4I03rI8H0BNTt2sXuX7iRODZZ004oQ8TO6tijFBSYncERFE98IBuzBVu0SKd7yorrYulptVOtKS+bZtJSf2XXwL3o1188Cg2d6yN3btDe4O0aOH6kgB5V25u7Ja5NeObV1WZVwevVPze/KZ+dJo3D9w/4wwTT+QsaZXYReQ8EVknIlUikvDngWdE6w3CxE4Op1To5aFgGRk6sY8ebdz5Cgr0MW1L6j6WblXMWgBjAHxiQCzukhf2PfbnP9sTB1EKSkvjN+R66y2djGtuqVTV1PwqqLmFf0TC9zU9qd9yS+D+t9+afDJnSasqRilVCCQeutSTli8HglssLFxoXyxEKajpjVpZmXiqubSnogtjZpVPhDvvDNzv1MmikzoDL57WVrR3fHiDXCIHy8y0qOSMwHksS+o+n+UsYWIXkQ9EZG2UW0q1cSIySUTyRSS/uLi49hE7ycqVoctBExoTuUlN4jWylUzNMW2pR2/UKHDfh50IEyZ2pdQQpVT3KLc3UzmRUmqOUipPKZXXsmXL2kfsJNEqEV96yfo4iAySkRGakJXSnZyCpmmNcP31eoya8MfZJvzkPqwqZlVMuq66KnT54ovtiYPIJPXqAXfdFZm4a2733aerdRwjuBnOjh32xWGjdJs7/klEigAMAPCOiPhvhudo07jdfbf1cRBRZGm9VSt74rBZWoldKbVIKdVOKZWllDpSKfUHowJzDZHIGYVvuokNdInsEFxaX7zYvjhsxqoYI3z5ZeS6Pn2sj4PIz8ILU8OH2xOHAzCxGyEjAzjmmNB1q1f7ZohQIkcILq2/+qp9cTgAE7tRovVsa9zY+jiI/Gj//tDl88+3Jw6HYGI3ighwwQWR6995x/pYiPwmJydwf8UK++JwCCZ2I738cuS6UaN82UGCyDLh1S79+9sTh4MwsRtJJHpyd1QjXyKPGTs2cD94GjwfY2I3WvCbLNgn/hsAk8h04b1KeV0LABO7OX7+OXLdwIG63zURGeOnn0KX2XfkMCZ2Mxx5JNC6deT6NCYmJqIwwXPsBY+9TkzspgkvTdS45x5r4yDyovAqmNtvtycOh2JiN4sIsGpV5PpZs4CiIuvjIfKK8J7ehw7ZE4eDMbGbqXfv6Ovbt7d2engir1AqdLiOc89lFWcUTOxmi9WG3eg5x4j8IHxm7NdesycOh2NiN5sIsGVL9G2DB1sbC5GbNW8eusxfvTExsVshNxeYOjVy/YcfcsYlomR8+inwyy+B5cWLI0vvdJgoG9p+5uXlqfz8fMvPa7tYU3StWQP06GFtLERuUVoKNGgQus6nbdZFpEApFWVOzlD8yrNSrA5KPXsCO3daGwuRGyjFpF4LTOxWysyM3b79yCOBAwesjYfI6cKrWzigXlKY2K3Wpg2wYEH0bQ0bsk0uUY3wqsutW2NXZ1IIJnY7nHcecM450bdlZXFMGaLwBD57dugQAhQXE7tdFi6Mva1uXf7kJP9q1ix0+ZhjgClT7InFpZjY7RQveWdmsuRO/tO9O/Drr6HrNm2yJxYXSyuxi8j9IrJeRNaIyCIROcKowHxBJH4ni7p1gfJy6+IhstOgQcC6daHr2AKmVtItsS8B0F0p1RPARgA3pB+Sz2RkAGVlsbfXqwccPGhdPER2OPts4KOPQtcxqddaWoldKfW+UqqmvuBzALy6URv16kXOsh6sfv3QXndEXtK/P/Dmm6HrmNTTYmQd+0QAi2NtFJFJIpIvIvnFxcUGntYjGjSIP19j8+bAt99aFw+RFUSAlStD1zGppy1hYheRD0RkbZTb6KB9bgRQAWBerOMopeYopfKUUnktW7Y0Jnqvadw4fnLv0kWPkUHkBdHapLM1mCESjh2rlBoSb7uIjAcwCsAZyo6BZ7ymcWNg3z4gJyf69hEjgJkzORMTuVu0pM70YZh0W8UMBzADwB+VUuwPb5SGDfXAR7Hcey+Qnc0PArlPeTmTugXSrWN/AkAjAEtEZLWIPG1ATAToxB2vHXtZmW5RE++iK5GTbNyoGwqEY1I3XLqtYjoppdorpXpV3yYbFRhBd1JKVOeYkxN58YnIaWbMAI49NnI9k7op2PPU6UT0m//UU2Pv078/MGGCZSERpUQEuO++0HU33cSkbiImdrf49FPg8cdjb3/hBf0BYtUMOcWBA9Hr01euBO680/p4fISJ3U2mTgXWro2/T04O8Pbb1sRDFMucOboRQLjffgPyEk4ARGlK2NyRHKZbt/jNIQHgrLP030OH9HgzRFZRKvZcpFVVHE/dIiyxu1HDhvpD0qFD/P3q1dMTZhNZobAwelIfM0YnfCZ1yzCxu5UI8MMPwKJF8fcbPFjvG2+gMaJ01CTtrl0jtxUWAq+/bn1MPsfE7nZnnx05fnU02dm63pPISMuWxa96Oe44a+MhAEzs3nDEEbrUdN558fe7/HJdstq61Zq4yLsqK/V76bTTIrfdfTerXmzPakzOAAAIa0lEQVTGxO4lCxYAmzcn3q9DBz39GKtnqDYuuwyoE6PdxYEDwKxZ1sZDEZjYvaZjR/0TePDg+Pv9+quunpk6lR1FKDkrVuhS+LPPRm677DL9Pqpf3/q4KAITuxeJAEuX6ouriTz5pK4jffFF8+Mid/rlF/2eOumk6NsPHAD+53+sjYniYmL3sg4ddCnq7rsT7zt+vP7wsnMT1Sgt1e+J5s2jb1+4kKV0h2Ji94NZs3SpKhlnnaU/zG+9ZW5M5FwVFfo90KBB7H2qqoBzzrEuJkoJE7tf1K+vS1cbNya3/+jR+sP9/POsg/eLmhJ6vN7Ke/awxYsLMLH7TefO+oOZqGNTjYkTdR38lCnxx4cn9yopSVxCX71av28aN7YuLqo1Jna/Ovts/XM62VH2nn5al+SOOEInAnK/zz/XCb1Fi9j7vPiiTugnnGBdXJQ2JnY/E9HjYldVAddem9xj9uzRiUAE+Oc/WU3jNlVVwLhx+vUbMCD2fo88ol/biy6yLjYyDBM76Q/5gw/q3oTTpiX/uD/9SVfTdOrEUrzTbdqkX+fMTGDevNj73X+/TuhXX21dbGQ4JnYKyMgAHn5Yl+oefjj5x333XaAUf8MNwMGD5sVIySsrA/r00a9L587x9339dZ3Qp0+3JjYyFRM7RRLRJXelgA8+SO2x996rW+CI6C+H8nJzYqToKir0ayeiexZ/+WX8/det06/zmDHWxEeWYGKn+M44Q3/wf/op9cdee60eE14EuO46PXsOGe/QIeCvfw00VXz00cSP2btXv67Rhtol10srsYvInSKyRkRWi8j7ItLGqMDIYdq00YmgogK4667UH//QQ0CTJjr5tG8PrFnDC6/p2Lo18MsoKwuYPTvxY556Sv/PlYo/Axe5Xrol9vuVUj2VUr0AvA3gFgNiIifLzARuvFEnhx9/1CXyVBUV6eZzGRk6MV14IbBtm/Gxesm+ffqCpoi+deiQ3LWM+vUDnYomTzY/TnKEtBK7Uir4t3VDACyC+Un79voCnVLAJ5/U/jgvvwy0bRtIWi++mPwQCF61bRswdmzgf9KoEfDYY8k//uuv9ety4AA7FflQ2nXsInK3iGwFcCFYYvev007TiaSqKv1xZsaP1/O61iS1zp314GT79xsTq9MUFelfQTXPV0R/0b36amrHWbEiUNXSvbs5sZIriEpQzykiHwA4KsqmG5VSbwbtdwOAbKXUrTGOMwnAJADo0KFD3x+SGVKW3C8/H+jXz5xjjxwJXHGFPn5Nc0unqqoCtmwB3nwT+NvfdJ8BI2zYAHTpYsyxyPFEpEAplZdwv0SJPYUT/g7AO0qphEWFvLw8lZ+fb8h5yUV27wZuvTW1KoV0jBwJDBsG5OXpTlRNm8Yf4Kq2KiqAnTuBtWuBTz/Vw9muX2/8eQDg+uuBO+7QTRnJdyxJ7CLSWSn1bfX9KwEMVEqdm+hxTOwEQPeGnDULeO01uyNxrnHjdPPFZs3sjoQcINnEnm4d+70islZE1gAYBoD9kCl5nTrpeVpr6oV37wYef9zuqOz13HO6BUzN/+Sll5jUKWXptoo5RynVvbrJ41lKqVr0YiGq1qRJYA7WmtvPP+sSqxlVKHbKztYXR/fvD32+l1yiLxwTpYE9T8nZjjwSuOoq3bsyOAEWF+vRJZ3eFf6ii4CPPtLNDoPjLy0Fzj8//hjoRLVUx+4AiGqlRQs9y9Po0dG3Hzqkk//GjUBhIbBqFbBype7xmq4+ffQF2d69gV69gNxcPS+o135VkGsxsZM31aun24K3bQsMGmR3NESWYlUMEZHHMLETEXkMEzsRkccwsRMReQwTOxGRxzCxExF5DBM7EZHHMLETEXmMYcP2pnRSkWIAfhmQvQWAXXYHYSG/PV+Az9kPnPJ8f6eUaploJ1sSu5+ISH4yw2x6hd+eL8Dn7Adue76siiEi8hgmdiIij2FiN98cuwOwmN+eL8Dn7Aeuer6sYyci8hiW2ImIPIaJ3QIicqWIbBCRdSJyn93xWEVEpouIEpEWdsdiNhG5X0TWi8gaEVkkIkfYHZMZRGR49Xt5k4jMtDses4lIexH5UEQKqz+/rpjXmYndZCIyCMBoAD2VUt0APGBzSJYQkfYAhgL40e5YLLIEQHelVE8AGwHcYHM8hhORTABPAjgTQFcAY0Wkq71Rma4CwHVKqeMBnATgr254zkzs5psC4F6lVBkAKKV22hyPVR4G8DcAvriIo5R6XylVUb34OYB2dsZjkv4ANimlNiulDgF4FbrQ4llKqe1KqVXV9/cCKATQ1t6oEmNiN18XAKeJyAoR+VhE+tkdkNlE5I8AflJKfWV3LDaZCGCx3UGYoC2ArUHLRXBBkjOKiOQC6A1ghb2RJMY5Tw0gIh8AOCrKphuh/8dNoX/G9QOwQESOVi5vjpTgOc8CMMzaiMwX7zkrpd6s3udG6J/v86yMzSISZZ2r38fJEpEcAK8DmKaU+s3ueBJhYjeAUmpIrG0iMgXAG9WJ/AsRqYIed6LYqvjMEOs5i0gPAB0BfCUigK6SWCUi/ZVSP1sYouHivc4AICLjAYwCcIbbv7hjKALQPmi5HYBtNsViGRGpC53U5yml3rA7nmSwKsZ8/wQwGABEpAuAenDGYEKmUEp9rZRqpZTKVUrlQieDPm5P6omIyHAAMwD8USl1wO54TLISQGcR6Sgi9QBcAOAtm2MylejSybMACpVSD9kdT7KY2M33HICjRWQt9MWm8R4tzfndEwAaAVgiIqtF5Gm7AzJa9cXhqQDeg76IuEAptc7eqEx3CoCLAAyufl1Xi8gIu4NKhD1PiYg8hiV2IiKPYWInIvIYJnYiIo9hYici8hgmdiIij2FiJyLyGCZ2IiKPYWInIvKY/wcyb2ZdsnrIogAAAABJRU5ErkJggg==\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "plot.plot(x_results[:,0,0], x_results[:,0,1], label='sun')\n",
    "plot.plot(x_results[:,1,0], x_results[:,1,1], 'b', label='earth')\n",
    "plot.plot(x_results[:,2,0], x_results[:,2,1], 'r', label='jupiter')\n",
    "plot.legend()\n",
    "plot.axis('equal')\n",
    "plot.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Energy energy results"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 25,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    },
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "time = np.linspace(0.0, tfinal, num=nsteps, endpoint=False)/(2.0*math.pi) # in years\n",
    "plot.plot(time, E_results[:,0], label='Kinetic energy')\n",
    "plot.plot(time, E_results[:,1], label='Potential energy')\n",
    "plot.plot(time, E_results[:,2], label='Total energy')\n",
    "plot.legend()\n",
    "plot.xlabel('time (years)')\n",
    "plot.ylabel('Energy')\n",
    "plot.show()\n",
    "plot.plot(time, (E_results[:,2]-E_results[0,2])/E_results[0,2], label='Total energy change')\n",
    "plot.xlabel('time (years)')\n",
    "plot.ylabel('Fractional Energy error')\n",
    "plot.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "collapsed": true
   },
   "source": [
    "So nothing much happens in this case except for a little precession of Jupiter."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [default]",
   "language": "python",
   "name": "python2"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 2
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython2",
   "version": "2.7.16"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 1
}
